# ANALYSIS FOR TEXTUALVERIFIER ONLY

- Comparative analysis for 5 different number variants of TextualVerifier
- Author: Eugenius Mario Situmorang
- Date: June 2025

In [ ]:
# %%
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import json
import re
from scipy import stats
from scipy.stats import fisher_exact
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

print("📊 All libraries imported successfully!")
print("🎯 Ready for multi-version comparative analysis")

## DATA LOADING AND PREPROCESSING FOR MULTIPLE VERSIONS

In [ ]:
def load_multiple_versions():
    """
    Load all 5 versions of verification data
    """
    print("🔄 Loading multiple versions...")
    
    # Define file paths and version names
    version_files = {
        "Version 1": "results/textualverifier-1v.csv",
        "Version 2": "results/textualverifier-2v.csv", 
        "Version 3": "results/textualverifier-3v.csv",
        "Version 4": "results/textualverifier-4v.csv",
        "Version 5": "results/textualverifier-5v.csv"
    }
    
    dataframes = {}
    
    for version_name, file_path in version_files.items():
        try:
            df = pd.read_csv(file_path)
            
            # Add version identifier
            df['version'] = version_name
            
            # Basic preprocessing
            df['success'] = df['success'].astype(bool) if 'success' in df.columns else True
            
            # Handle answer correctness columns
            if 'original_answer_correctness' in df.columns:
                df['original_answer_correctness'] = df['original_answer_correctness'].str.strip().str.upper() == 'TRUE'
            if 'verifier_answer_correctness' in df.columns:
                df['verifier_answer_correctness'] = df['verifier_answer_correctness'].str.strip().str.upper() == 'TRUE'
            
            # Parse rating arrays
            def parse_rating_array(rating_str):
                try:
                    if pd.isna(rating_str):
                        return []
                    rating_str = str(rating_str).strip('[]')
                    return [int(x.strip()) for x in rating_str.split(',') if x.strip()]
                except:
                    return []
            
            if 'original_rating' in df.columns:
                df['original_rating_parsed'] = df['original_rating'].apply(parse_rating_array)
                df['original_avg_rating'] = df['original_rating_parsed'].apply(lambda x: np.mean(x) if x else np.nan)
            
            if 'verified_rating' in df.columns:
                df['verified_rating_parsed'] = df['verified_rating'].apply(parse_rating_array)
                df['verified_avg_rating'] = df['verified_rating_parsed'].apply(lambda x: np.mean(x) if x else np.nan)
            
            # Calculate derived metrics
            if 'original_avg_rating' in df.columns and 'verified_avg_rating' in df.columns:
                df['rating_improvement'] = df['verified_avg_rating'] - df['original_avg_rating']
            
            if 'verified_total_steps' in df.columns and 'original_total_steps' in df.columns:
                df['step_change'] = df['verified_total_steps'] - df['original_total_steps']
            
            if 'total_output_tokens' in df.columns and 'total_input_tokens' in df.columns:
                df['token_efficiency'] = df['total_output_tokens'] / (df['total_input_tokens'] + 1)
            
            dataframes[version_name] = df
            print(f"✅ {version_name}: {len(df)} records loaded")
            
        except Exception as e:
            print(f"❌ Error loading {version_name}: {e}")
            continue
    
    # Combine all dataframes
    if dataframes:
        combined_df = pd.concat(dataframes.values(), ignore_index=True)
        print(f"\n🎯 Total combined dataset: {len(combined_df)} records across {len(dataframes)} versions")
        return combined_df, dataframes
    else:
        print("❌ No data loaded successfully")
        return None, {}

# Load all versions
combined_df, version_dfs = load_multiple_versions()

## MULTI-VERSION COMPARATIVE ANALYSIS

In [ ]:
def comparative_foundational_analysis(combined_df):
    """
    Compare foundational metrics across all versions
    """
    print("\n" + "="*70)
    print("📈 COMPARATIVE FOUNDATIONAL ANALYSIS")
    print("="*70)
    
    results = {}
    
    # Group by version for analysis
    version_stats = combined_df.groupby('version').agg({
        'success': ['count', 'mean'],
        'original_answer_correctness': 'mean',
        'verifier_answer_correctness': 'mean', 
        'processing_time_ms': ['mean', 'median', 'std'],
        'total_llm_calls': ['mean', 'median'],
        'total_input_tokens': 'mean',
        'total_output_tokens': 'mean',
        'token_efficiency': ['mean', 'std']
    }).round(3)
    
    # Flatten column names
    version_stats.columns = ['_'.join(col).strip() for col in version_stats.columns]
    
    # Calculate accuracy improvements
    accuracy_improvements = {}
    for version in combined_df['version'].unique():
        version_data = combined_df[combined_df['version'] == version]
        original_acc = version_data['original_answer_correctness'].mean() * 100
        verified_acc = version_data['verifier_answer_correctness'].mean() * 100
        accuracy_improvements[version] = verified_acc - original_acc
    
    results['version_stats'] = version_stats
    results['accuracy_improvements'] = accuracy_improvements
    
    # Print comparison summary
    print("📊 VERSION COMPARISON SUMMARY:")
    print("-" * 50)
    
    for version in combined_df['version'].unique():
        version_data = combined_df[combined_df['version'] == version]
        success_rate = version_data['success'].mean() * 100
        accuracy_improvement = accuracy_improvements[version]
        avg_time = version_data['processing_time_ms'].mean()
        avg_llm_calls = version_data['total_llm_calls'].mean()
        
        print(f"\n{version}:")
        print(f"  • Success Rate: {success_rate:.1f}%")
        print(f"  • Accuracy Improvement: {accuracy_improvement:+.1f} pp")
        print(f"  • Avg Processing Time: {avg_time:.1f} ms")
        print(f"  • Avg LLM Calls: {avg_llm_calls:.1f}")
    
    # Statistical significance tests
    print(f"\n🔬 STATISTICAL SIGNIFICANCE TESTS:")
    print("-" * 40)
    
    versions = list(combined_df['version'].unique())
    
    # Compare success rates using Fisher's exact test (more robust than chi-square)
    if len(versions) >= 2:
        for i in range(len(versions)):
            for j in range(i+1, len(versions)):
                v1_success = combined_df[combined_df['version'] == versions[i]]['success']
                v2_success = combined_df[combined_df['version'] == versions[j]]['success']
                
                if len(v1_success) > 0 and len(v2_success) > 0:
                    try:
                        # Create contingency table
                        v1_successes = v1_success.sum()
                        v1_failures = len(v1_success) - v1_successes
                        v2_successes = v2_success.sum()
                        v2_failures = len(v2_success) - v2_successes
                        
                        contingency_table = [[v1_successes, v1_failures], 
                                           [v2_successes, v2_failures]]
                        
                        # Use Fisher's exact test for small samples or when chi-square assumptions violated
                        if any(cell < 5 for row in contingency_table for cell in row):
                            odds_ratio, p_value = fisher_exact(contingency_table)
                            test_used = "Fisher's exact"
                        else:
                            # Check if any expected frequencies are zero
                            total = sum(sum(row) for row in contingency_table)
                            row_totals = [sum(row) for row in contingency_table]
                            col_totals = [sum(contingency_table[i][j] for i in range(2)) for j in range(2)]
                            
                            expected_freq_valid = all(
                                (row_totals[i] * col_totals[j]) / total >= 1
                                for i in range(2) for j in range(2)
                            )
                            
                            if expected_freq_valid:
                                stat, p_value = stats.chi2_contingency(contingency_table)[:2]
                                test_used = "Chi-square"
                            else:
                                odds_ratio, p_value = fisher_exact(contingency_table)
                                test_used = "Fisher's exact"
                        
                        significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
                        print(f"  Success Rate {versions[i]} vs {versions[j]}: p={p_value:.4f} {significance} ({test_used})")
                        
                    except Exception as e:
                        # Fallback to simple proportion comparison
                        v1_rate = v1_success.mean()
                        v2_rate = v2_success.mean()
                        rate_diff = abs(v1_rate - v2_rate)
                        print(f"  Success Rate {versions[i]} vs {versions[j]}: Rate diff={rate_diff:.3f} (comparison failed: {str(e)[:50]}...)")
    
    return results

# Run comparative foundational analysis
comp_results_1 = comparative_foundational_analysis(combined_df)

In [ ]:
# Visualization for Multi-Version Comparison
def visualize_version_comparison(combined_df):
    """
    Create comprehensive visualizations comparing all versions
    """
    fig, axes = plt.subplots(3, 3, figsize=(20, 15))
    fig.suptitle('🔍 Multi-Version Comparison Dashboard', fontsize=18, fontweight='bold')
    
    versions = combined_df['version'].unique()
    colors = plt.cm.Set2(np.linspace(0, 1, len(versions)))
    
    # 1. Success Rate Comparison
    success_rates = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        success_rates.append(version_data['success'].mean() * 100)
    
    bars1 = axes[0,0].bar(versions, success_rates, color=colors)
    axes[0,0].set_ylabel('Success Rate (%)')
    axes[0,0].set_title('Success Rate by Version')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    for bar, value in zip(bars1, success_rates):
        axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                      f'{value:.1f}%', ha='center', va='bottom')
    
    # 2. Accuracy Improvement Comparison  
    accuracy_improvements = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        original_acc = version_data['original_answer_correctness'].mean() * 100
        verified_acc = version_data['verifier_answer_correctness'].mean() * 100
        accuracy_improvements.append(verified_acc - original_acc)
    
    bars2 = axes[0,1].bar(versions, accuracy_improvements, color=colors)
    axes[0,1].set_ylabel('Accuracy Improvement (pp)')
    axes[0,1].set_title('Accuracy Improvement by Version')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    
    for bar, value in zip(bars2, accuracy_improvements):
        axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + (0.1 if value >= 0 else -0.3), 
                      f'{value:+.1f}', ha='center', va='bottom' if value >= 0 else 'top')
    
    # 3. Processing Time Distribution
    processing_times_by_version = []
    labels = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        times = version_data['processing_time_ms'].dropna()
        if len(times) > 0:
            processing_times_by_version.append(times)
            labels.append(version)
    
    if processing_times_by_version:
        bp1 = axes[0,2].boxplot(processing_times_by_version, labels=labels, patch_artist=True)
        for patch, color in zip(bp1['boxes'], colors):
            patch.set_facecolor(color)
    axes[0,2].set_ylabel('Processing Time (ms)')
    axes[0,2].set_title('Processing Time Distribution')
    axes[0,2].tick_params(axis='x', rotation=45)
    
    # 4. LLM Calls Comparison
    llm_calls_avg = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        llm_calls_avg.append(version_data['total_llm_calls'].mean())
    
    bars3 = axes[1,0].bar(versions, llm_calls_avg, color=colors)
    axes[1,0].set_ylabel('Average LLM Calls')
    axes[1,0].set_title('LLM Calls by Version')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    for bar, value in zip(bars3, llm_calls_avg):
        axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                      f'{value:.1f}', ha='center', va='bottom')
    
    # 5. Token Efficiency Comparison
    token_efficiency_avg = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        token_efficiency_avg.append(version_data['token_efficiency'].mean())
    
    bars4 = axes[1,1].bar(versions, token_efficiency_avg, color=colors)
    axes[1,1].set_ylabel('Average Token Efficiency')
    axes[1,1].set_title('Token Efficiency by Version')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    for bar, value in zip(bars4, token_efficiency_avg):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                      f'{value:.3f}', ha='center', va='bottom')
    
    # 6. Rating Improvement Comparison
    rating_improvements = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        rating_imp = version_data['rating_improvement'].dropna().mean()
        rating_improvements.append(rating_imp if not pd.isna(rating_imp) else 0)
    
    bars5 = axes[1,2].bar(versions, rating_improvements, color=colors)
    axes[1,2].set_ylabel('Average Rating Improvement')
    axes[1,2].set_title('Rating Improvement by Version')
    axes[1,2].tick_params(axis='x', rotation=45)
    axes[1,2].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    
    for bar, value in zip(bars5, rating_improvements):
        axes[1,2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + (0.01 if value >= 0 else -0.02), 
                      f'{value:+.3f}', ha='center', va='bottom' if value >= 0 else 'top')
    
    # 7. Step Change Comparison
    step_changes = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        step_change = version_data['step_change'].dropna().mean()
        step_changes.append(step_change if not pd.isna(step_change) else 0)
    
    bars6 = axes[2,0].bar(versions, step_changes, color=colors)
    axes[2,0].set_ylabel('Average Step Change')
    axes[2,0].set_title('Step Change by Version')
    axes[2,0].tick_params(axis='x', rotation=45)
    axes[2,0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    
    for bar, value in zip(bars6, step_changes):
        axes[2,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + (0.1 if value >= 0 else -0.2), 
                      f'{value:+.1f}', ha='center', va='bottom' if value >= 0 else 'top')
    
    # 8. Performance Efficiency Scatter (Time vs Token Efficiency)
    for i, version in enumerate(versions):
        version_data = combined_df[combined_df['version'] == version]
        valid_data = version_data.dropna(subset=['processing_time_ms', 'token_efficiency'])
        if len(valid_data) > 0:
            axes[2,1].scatter(valid_data['processing_time_ms'], valid_data['token_efficiency'], 
                            label=version, alpha=0.6, color=colors[i])
    
    axes[2,1].set_xlabel('Processing Time (ms)')
    axes[2,1].set_ylabel('Token Efficiency')
    axes[2,1].set_title('Performance Efficiency by Version')
    axes[2,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 9. Overall Performance Radar Chart (simplified)
    metrics = ['Success Rate', 'Accuracy Imp', 'Speed', 'Efficiency', 'Rating Imp']
    
    # Normalize metrics to 0-100 scale for comparison
    normalized_data = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        
        # Normalize each metric
        success_norm = version_data['success'].mean() * 100
        acc_imp_norm = max(0, (version_data['verifier_answer_correctness'].mean() - 
                              version_data['original_answer_correctness'].mean()) * 100 + 50)
        speed_norm = max(0, 100 - (version_data['processing_time_ms'].mean() / 5))  # Inverse for speed
        efficiency_norm = min(100, version_data['token_efficiency'].mean() * 100)
        rating_norm = max(0, version_data['rating_improvement'].dropna().mean() * 50 + 50)
        
        normalized_data.append([success_norm, acc_imp_norm, speed_norm, efficiency_norm, rating_norm])
    
    # Create a simple bar chart instead of radar for better visibility
    x_pos = np.arange(len(metrics))
    width = 0.15
    
    for i, (version, data) in enumerate(zip(versions, normalized_data)):
        axes[2,2].bar(x_pos + i * width, data, width, label=version, color=colors[i], alpha=0.8)
    
    axes[2,2].set_xlabel('Metrics')
    axes[2,2].set_ylabel('Normalized Score (0-100)')
    axes[2,2].set_title('Overall Performance Comparison')
    axes[2,2].set_xticks(x_pos + width * 2)
    axes[2,2].set_xticklabels(metrics, rotation=45, ha='right')
    axes[2,2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

visualize_version_comparison(combined_df)

## DETAILED VERSION RANKING ANALYSIS

In [ ]:
def comprehensive_version_ranking(combined_df):
    """
    Create comprehensive ranking system for all versions
    """
    print("\n" + "="*70)
    print("🏆 COMPREHENSIVE VERSION RANKING ANALYSIS")
    print("="*70)
    
    versions = combined_df['version'].unique()
    ranking_data = {}
    
    # Define metrics and their weights
    metrics = {
        'success_rate': {'weight': 0.25, 'higher_better': True},
        'accuracy_improvement': {'weight': 0.25, 'higher_better': True},
        'processing_speed': {'weight': 0.20, 'higher_better': True},  # Inverse of time
        'token_efficiency': {'weight': 0.15, 'higher_better': True},
        'rating_improvement': {'weight': 0.15, 'higher_better': True}
    }
    
    # Calculate metrics for each version
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        
        # Success rate (0-100)
        success_rate = version_data['success'].mean() * 100
        
        # Accuracy improvement (-100 to +100)
        accuracy_improvement = (version_data['verifier_answer_correctness'].mean() - 
                               version_data['original_answer_correctness'].mean()) * 100
        
        # Processing speed (inverse of time, normalized)
        avg_time = version_data['processing_time_ms'].mean()
        processing_speed = 1000 / avg_time if avg_time > 0 else 0  # Operations per second
        
        # Token efficiency
        token_efficiency = version_data['token_efficiency'].mean()
        
        # Rating improvement
        rating_improvement = version_data['rating_improvement'].dropna().mean()
        if pd.isna(rating_improvement):
            rating_improvement = 0
        
        ranking_data[version] = {
            'success_rate': success_rate,
            'accuracy_improvement': accuracy_improvement,
            'processing_speed': processing_speed,
            'token_efficiency': token_efficiency,
            'rating_improvement': rating_improvement,
            'avg_processing_time': avg_time,
            'total_problems': len(version_data)
        }
    
    # Normalize metrics to 0-100 scale
    normalized_data = {}
    
    for metric in metrics.keys():
        values = [ranking_data[version][metric] for version in versions]
        
        if metrics[metric]['higher_better']:
            min_val, max_val = min(values), max(values)
            if max_val > min_val:
                for version in versions:
                    if version not in normalized_data:
                        normalized_data[version] = {}
                    normalized_data[version][metric] = ((ranking_data[version][metric] - min_val) / 
                                                       (max_val - min_val)) * 100
            else:
                for version in versions:
                    if version not in normalized_data:
                        normalized_data[version] = {}
                    normalized_data[version][metric] = 100
    
    # Calculate weighted scores
    final_scores = {}
    for version in versions:
        total_score = 0
        for metric, config in metrics.items():
            total_score += normalized_data[version][metric] * config['weight']
        final_scores[version] = total_score
    
    # Create ranking
    sorted_versions = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Print detailed ranking
    print("🥇 FINAL RANKING:")
    print("=" * 50)
    
    for rank, (version, score) in enumerate(sorted_versions, 1):
        medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else f"{rank}."
        print(f"\n{medal} {version} - Overall Score: {score:.1f}/100")
        print(f"   Success Rate: {ranking_data[version]['success_rate']:.1f}%")
        print(f"   Accuracy Improvement: {ranking_data[version]['accuracy_improvement']:+.1f} pp")
        print(f"   Avg Processing Time: {ranking_data[version]['avg_processing_time']:.1f} ms")
        print(f"   Token Efficiency: {ranking_data[version]['token_efficiency']:.3f}")
        print(f"   Rating Improvement: {ranking_data[version]['rating_improvement']:+.3f}")
        print(f"   Total Problems: {ranking_data[version]['total_problems']}")
    
    # Detailed metric breakdown
    print(f"\n📊 DETAILED METRIC BREAKDOWN:")
    print("=" * 50)
    
    metric_rankings = {}
    for metric in metrics.keys():
        metric_scores = [(version, normalized_data[version][metric]) for version in versions]
        metric_scores.sort(key=lambda x: x[1], reverse=True)
        metric_rankings[metric] = metric_scores
        
        print(f"\n{metric.replace('_', ' ').title()}:")
        for rank, (version, score) in enumerate(metric_scores, 1):
            print(f"   {rank}. {version}: {score:.1f}/100")
    
    return {
        'final_ranking': sorted_versions,
        'metric_rankings': metric_rankings,
        'raw_data': ranking_data,
        'normalized_data': normalized_data
    }

# Run comprehensive ranking
ranking_results = comprehensive_version_ranking(combined_df)

## CORRELATION HEATMAP ANALYSIS (14x14)

In [ ]:
def create_correlation_heatmap_analysis(combined_df):
    """
    Create comprehensive 14x14 correlation heatmap analysis
    """
    print("\n" + "="*70)
    print("🔥 CORRELATION HEATMAP ANALYSIS (14x14)")
    print("="*70)
    
    # Define the 14 columns for correlation analysis
    correlation_columns = [
        'original_answer_correctness',
        'original_total_steps', 
        'original_neg1',
        'original_zero',
        'original_pos1',
        'verifier_answer_correctness',
        'verified_total_steps',
        'verifier_neg1', 
        'verifier_zero',
        'verifier_pos1',
        'processing_time_ms',
        'total_llm_calls',
        'total_input_tokens',
        'total_output_tokens'
    ]
    
    # Check which columns exist in the dataset
    available_columns = []
    missing_columns = []
    
    for col in correlation_columns:
        if col in combined_df.columns:
            available_columns.append(col)
        else:
            missing_columns.append(col)
    
    print(f"📊 Available columns: {len(available_columns)}/{len(correlation_columns)}")
    if missing_columns:
        print(f"⚠️  Missing columns: {missing_columns}")
    
    # Prepare data for correlation analysis
    correlation_data = combined_df[available_columns].copy()
    
    # Convert boolean columns to numeric
    bool_columns = ['original_answer_correctness', 'verifier_answer_correctness']
    for col in bool_columns:
        if col in correlation_data.columns:
            correlation_data[col] = correlation_data[col].astype(int)
    
    # Calculate correlation matrix
    correlation_matrix = correlation_data.corr()
    
    # Create the heatmap visualization
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    fig.suptitle('🔥 Comprehensive Correlation Analysis (14x14)', fontsize=18, fontweight='bold')
    
    # 1. Main correlation heatmap (top left)
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
    sns.heatmap(correlation_matrix, 
                annot=True, 
                fmt='.3f',
                cmap='RdBu_r',
                center=0,
                square=True,
                mask=mask,
                cbar_kws={'label': 'Correlation Coefficient'},
                ax=axes[0,0])
    axes[0,0].set_title('Correlation Heatmap (Lower Triangle)', fontweight='bold')
    axes[0,0].tick_params(axis='x', rotation=45, labelsize=9)
    axes[0,0].tick_params(axis='y', rotation=0, labelsize=9)
    
    # 2. Full correlation heatmap without mask (top right)
    sns.heatmap(correlation_matrix,
                annot=False,  # No annotations for better visibility
                cmap='RdBu_r',
                center=0,
                square=True,
                cbar_kws={'label': 'Correlation Coefficient'},
                ax=axes[0,1])
    axes[0,1].set_title('Full Correlation Heatmap', fontweight='bold')
    axes[0,1].tick_params(axis='x', rotation=45, labelsize=9)
    axes[0,1].tick_params(axis='y', rotation=0, labelsize=9)
    
    # 3. Correlation strength distribution (bottom left)
    correlation_values = correlation_matrix.values
    correlation_flat = correlation_values[np.triu_indices_from(correlation_values, k=1)]  # Upper triangle excluding diagonal
    
    axes[1,0].hist(correlation_flat, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1,0].axvline(0, color='red', linestyle='--', alpha=0.7, label='Zero correlation')
    axes[1,0].axvline(np.mean(correlation_flat), color='green', linestyle='--', alpha=0.7, 
                     label=f'Mean: {np.mean(correlation_flat):.3f}')
    axes[1,0].set_xlabel('Correlation Coefficient')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Distribution of Correlation Values')
    axes[1,0].legend()
    axes[1,0].grid(axis='y', alpha=0.3)
    
    # 4. Top correlations analysis (bottom right)
    axes[1,1].axis('off')
    
    # Find strongest positive and negative correlations
    correlation_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if not np.isnan(corr_value):
                correlation_pairs.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j], 
                    corr_value
                ))
    
    # Sort by absolute correlation value
    correlation_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
    
    # Display top correlations
    top_correlations_text = "🔝 STRONGEST CORRELATIONS:\n" + "="*40 + "\n\n"
    
    top_correlations_text += "📈 STRONGEST POSITIVE:\n"
    positive_corrs = [pair for pair in correlation_pairs if pair[2] > 0][:5]
    for i, (col1, col2, corr) in enumerate(positive_corrs, 1):
        col1_short = col1.replace('_', ' ').title()[:20]
        col2_short = col2.replace('_', ' ').title()[:20]
        top_correlations_text += f"{i}. {col1_short} ↔ {col2_short}\n   r = {corr:.3f}\n\n"
    
    top_correlations_text += "📉 STRONGEST NEGATIVE:\n"
    negative_corrs = [pair for pair in correlation_pairs if pair[2] < 0][:5]
    for i, (col1, col2, corr) in enumerate(negative_corrs, 1):
        col1_short = col1.replace('_', ' ').title()[:20]
        col2_short = col2.replace('_', ' ').title()[:20]
        top_correlations_text += f"{i}. {col1_short} ↔ {col2_short}\n   r = {corr:.3f}\n\n"
    
    # Calculate correlation strength categories
    strong_pos = len([pair for pair in correlation_pairs if pair[2] >= 0.7])
    moderate_pos = len([pair for pair in correlation_pairs if 0.3 <= pair[2] < 0.7])
    weak_pos = len([pair for pair in correlation_pairs if 0.1 <= pair[2] < 0.3])
    negligible = len([pair for pair in correlation_pairs if -0.1 < pair[2] < 0.1])
    weak_neg = len([pair for pair in correlation_pairs if -0.3 < pair[2] <= -0.1])
    moderate_neg = len([pair for pair in correlation_pairs if -0.7 < pair[2] <= -0.3])
    strong_neg = len([pair for pair in correlation_pairs if pair[2] <= -0.7])
    
    top_correlations_text += f"\n📊 CORRELATION STRENGTH SUMMARY:\n"
    top_correlations_text += f"Strong Positive (≥0.7): {strong_pos}\n"
    top_correlations_text += f"Moderate Positive (0.3-0.7): {moderate_pos}\n"
    top_correlations_text += f"Weak Positive (0.1-0.3): {weak_pos}\n"
    top_correlations_text += f"Negligible (-0.1 to 0.1): {negligible}\n"
    top_correlations_text += f"Weak Negative (-0.3 to -0.1): {weak_neg}\n"
    top_correlations_text += f"Moderate Negative (-0.7 to -0.3): {moderate_neg}\n"
    top_correlations_text += f"Strong Negative (≤-0.7): {strong_neg}\n"
    
    axes[1,1].text(0.05, 0.95, top_correlations_text, transform=axes[1,1].transAxes, 
                   fontsize=10, verticalalignment='top', fontfamily='monospace',
                   bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.3))
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed correlation analysis
    print(f"\n📋 DETAILED CORRELATION ANALYSIS:")
    print("="*50)
    
    print(f"📊 Dataset Statistics:")
    print(f"   • Total correlations calculated: {len(correlation_pairs)}")
    print(f"   • Mean correlation: {np.mean([pair[2] for pair in correlation_pairs]):.3f}")
    print(f"   • Std correlation: {np.std([pair[2] for pair in correlation_pairs]):.3f}")
    print(f"   • Max correlation: {max([pair[2] for pair in correlation_pairs]):.3f}")
    print(f"   • Min correlation: {min([pair[2] for pair in correlation_pairs]):.3f}")
    
    # Version-specific correlation analysis
    print(f"\n🔍 VERSION-SPECIFIC CORRELATION INSIGHTS:")
    for version in combined_df['version'].unique():
        version_data = combined_df[combined_df['version'] == version]
        version_corr_data = version_data[available_columns].copy()
        
        # Convert boolean columns to numeric for this version
        for col in bool_columns:
            if col in version_corr_data.columns:
                version_corr_data[col] = version_corr_data[col].astype(int)
        
        if len(version_corr_data) > 2:  # Need sufficient data for correlation
            version_corr_matrix = version_corr_data.corr()
            
            # Find strongest correlation for this version
            version_pairs = []
            for i in range(len(version_corr_matrix.columns)):
                for j in range(i+1, len(version_corr_matrix.columns)):
                    corr_value = version_corr_matrix.iloc[i, j]
                    if not np.isnan(corr_value):
                        version_pairs.append((
                            version_corr_matrix.columns[i],
                            version_corr_matrix.columns[j], 
                            corr_value
                        ))
            
            if version_pairs:
                strongest_corr = max(version_pairs, key=lambda x: abs(x[2]))
                mean_corr = np.mean([abs(pair[2]) for pair in version_pairs])
                print(f"   • {version}: Strongest |r|={abs(strongest_corr[2]):.3f}, Mean |r|={mean_corr:.3f}")
    
    return {
        'correlation_matrix': correlation_matrix,
        'correlation_pairs': correlation_pairs,
        'available_columns': available_columns,
        'missing_columns': missing_columns,
        'correlation_stats': {
            'mean': np.mean([pair[2] for pair in correlation_pairs]),
            'std': np.std([pair[2] for pair in correlation_pairs]),
            'max': max([pair[2] for pair in correlation_pairs]) if correlation_pairs else 0,
            'min': min([pair[2] for pair in correlation_pairs]) if correlation_pairs else 0
        }
    }

# Run correlation heatmap analysis
correlation_results = create_correlation_heatmap_analysis(combined_df)

In [ ]:
def statistical_analysis_and_recommendations(combined_df, ranking_results):
    """
    Perform statistical analysis and generate actionable recommendations
    """
    print("\n" + "="*70)
    print("📊 STATISTICAL ANALYSIS & RECOMMENDATIONS")
    print("="*70)
    
    versions = combined_df['version'].unique()
    
    # Statistical significance testing
    print("🔬 STATISTICAL SIGNIFICANCE TESTS:")
    print("-" * 50)
    
    # ANOVA tests for continuous variables
    continuous_vars = ['processing_time_ms', 'total_llm_calls', 'token_efficiency', 'rating_improvement']
    
    for var in continuous_vars:
        if var in combined_df.columns:
            groups = []
            group_names = []
            for version in versions:
                version_data = combined_df[combined_df['version'] == version][var].dropna()
                if len(version_data) > 1:  # Need at least 2 data points
                    groups.append(version_data)
                    group_names.append(version)
            
            if len(groups) >= 2:
                try:
                    # Check if there's any variance in the data
                    all_data = np.concatenate(groups)
                    if np.var(all_data) > 1e-10:  # Avoid division by zero
                        f_stat, p_value = stats.f_oneway(*groups)
                        significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
                        print(f"  {var}: F={f_stat:.3f}, p={p_value:.4f} {significance}")
                    else:
                        print(f"  {var}: No variance detected (all values approximately equal)")
                except Exception as e:
                    print(f"  {var}: ANOVA failed ({str(e)[:30]}...)")
            else:
                print(f"  {var}: Insufficient data for ANOVA (need ≥2 groups with ≥2 data points each)")
    
    # Effect size analysis (Cohen's d between best and worst versions)
    best_version = ranking_results['final_ranking'][0][0]
    worst_version = ranking_results['final_ranking'][-1][0]
    
    print(f"\n📏 EFFECT SIZE ANALYSIS ({best_version} vs {worst_version}):")
    print("-" * 50)
    
    def cohens_d(group1, group2):
        """Calculate Cohen's d effect size with error handling"""
        try:
            if len(group1) < 2 or len(group2) < 2:
                return np.nan
            
            n1, n2 = len(group1), len(group2)
            s1, s2 = group1.std(), group2.std()
            
            # Handle case where standard deviations are zero
            if s1 == 0 and s2 == 0:
                return 0.0 if group1.mean() == group2.mean() else np.inf
            
            pooled_std = np.sqrt(((n1-1)*s1**2 + (n2-1)*s2**2) / (n1+n2-2))
            
            if pooled_std == 0:
                return 0.0 if group1.mean() == group2.mean() else np.inf
            
            return (group1.mean() - group2.mean()) / pooled_std
        except Exception as e:
            return np.nan
    
    for var in continuous_vars:
        if var in combined_df.columns:
            best_data = combined_df[combined_df['version'] == best_version][var].dropna()
            worst_data = combined_df[combined_df['version'] == worst_version][var].dropna()
            
            if len(best_data) > 0 and len(worst_data) > 0:
                effect_size = cohens_d(best_data, worst_data)
                magnitude = ("Large" if abs(effect_size) >= 0.8 else 
                           "Medium" if abs(effect_size) >= 0.5 else "Small")
                print(f"  {var}: Cohen's d = {effect_size:.3f} ({magnitude})")
    
    # Confidence intervals for key metrics
    print(f"\n🎯 95% CONFIDENCE INTERVALS FOR KEY METRICS:")
    print("-" * 50)
    
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        print(f"\n{version}:")
        
        # Success rate CI
        n = len(version_data)
        p = version_data['success'].mean()
        se = np.sqrt(p * (1-p) / n)
        ci_lower = p - 1.96 * se
        ci_upper = p + 1.96 * se
        print(f"  Success Rate: {p*100:.1f}% [{ci_lower*100:.1f}%, {ci_upper*100:.1f}%]")
        
        # Processing time CI
        if 'processing_time_ms' in version_data.columns:
            times = version_data['processing_time_ms'].dropna()
            if len(times) > 1:
                mean_time = times.mean()
                se_time = times.std() / np.sqrt(len(times))
                ci_lower_time = mean_time - 1.96 * se_time
                ci_upper_time = mean_time + 1.96 * se_time
                print(f"  Processing Time: {mean_time:.1f}ms [{ci_lower_time:.1f}ms, {ci_upper_time:.1f}ms]")
    
    # Generate recommendations
    print(f"\n🎯 ACTIONABLE RECOMMENDATIONS:")
    print("="*50)
    
    recommendations = []
    
    # Best performer analysis
    best_version = ranking_results['final_ranking'][0][0]
    best_score = ranking_results['final_ranking'][0][1]
    
    recommendations.append(f"🥇 ADOPT BEST PRACTICES: {best_version} achieved the highest overall score ({best_score:.1f}/100)")
    
    # Identify strongest metrics per version
    metric_winners = {}
    for metric in ranking_results['metric_rankings']:
        winner = ranking_results['metric_rankings'][metric][0]
        metric_winners[metric] = winner
    
    print(f"\n🏆 METRIC CHAMPIONS:")
    for metric, (version, score) in metric_winners.items():
        print(f"  • {metric.replace('_', ' ').title()}: {version} ({score:.1f}/100)")
        
        if metric == 'processing_speed':
            recommendations.append(f"⚡ SPEED OPTIMIZATION: Learn from {version}'s processing optimizations")
        elif metric == 'accuracy_improvement':
            recommendations.append(f"🎯 ACCURACY FOCUS: Adopt {version}'s verification strategies")
        elif metric == 'token_efficiency':
            recommendations.append(f"💰 COST OPTIMIZATION: Implement {version}'s token efficiency techniques")
    
    # Identify improvement opportunities
    print(f"\n🔍 IMPROVEMENT OPPORTUNITIES:")
    
    for version in versions:
        version_scores = {metric: ranking_results['normalized_data'][version][metric] 
                         for metric in ranking_results['normalized_data'][version]}
        weakest_metric = min(version_scores, key=version_scores.get)
        weakest_score = version_scores[weakest_metric]
        
        if weakest_score < 50:  # Below average performance
            print(f"  • {version}: Improve {weakest_metric.replace('_', ' ')} (current: {weakest_score:.1f}/100)")
            recommendations.append(f"📈 {version} FOCUS: Prioritize {weakest_metric.replace('_', ' ')} improvements")
    
    # Version-specific insights
    print(f"\n💡 VERSION-SPECIFIC INSIGHTS:")
    
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        insights = []
        
        # Success rate insight
        success_rate = version_data['success'].mean() * 100
        if success_rate >= 95:
            insights.append("Very high reliability")
        elif success_rate < 80:
            insights.append("Reliability concerns")
        
        # Processing time insight
        avg_time = version_data['processing_time_ms'].mean()
        if avg_time < 50:
            insights.append("Fast processing")
        elif avg_time > 100:
            insights.append("Slow processing")
        
        # Token efficiency insight
        token_eff = version_data['token_efficiency'].mean()
        if token_eff > 0.3:
            insights.append("High token efficiency")
        elif token_eff < 0.15:
            insights.append("Low token efficiency")
        
        if insights:
            print(f"  • {version}: {', '.join(insights)}")
    
    # Final recommendations
    print(f"\n📋 FINAL RECOMMENDATIONS:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
    
    # Strategic recommendations
    strategic_recs = [
        "🔄 Implement A/B testing framework to continuously evaluate version performance",
        "📊 Set up monitoring dashboards to track key metrics in real-time", 
        "🎯 Focus development efforts on the weakest performing metrics",
        "🔬 Conduct detailed analysis of failure cases to improve reliability",
        "💰 Optimize token usage based on best-performing version's strategies"
    ]
    
    print(f"\n🚀 STRATEGIC RECOMMENDATIONS:")
    for i, rec in enumerate(strategic_recs, 1):
        print(f"{i}. {rec}")

    return {
        'recommendations': recommendations,
        'strategic_recommendations': strategic_recs,
        'metric_winners': metric_winners
    }

# Run statistical analysis and recommendations
analysis_recommendations = statistical_analysis_and_recommendations(combined_df, ranking_results)


## CORRELATION HEATMAP ANALYSIS (14x14)

In [ ]:
def create_correlation_heatmap_analysis(combined_df):
    """
    Create comprehensive 14x14 correlation heatmap analysis
    """
    print("\n" + "="*70)
    print("🔥 CORRELATION HEATMAP ANALYSIS (14x14)")
    print("="*70)
    
    # Define the 14 columns for correlation analysis
    correlation_columns = [
        'original_answer_correctness',
        'original_total_steps', 
        'original_neg1',
        'original_zero',
        'original_pos1',
        'verifier_answer_correctness',
        'verified_total_steps',
        'verifier_neg1', 
        'verifier_zero',
        'verifier_pos1',
        'processing_time_ms',
        'total_llm_calls',
        'total_input_tokens',
        'total_output_tokens'
    ]
    
    # Check which columns exist in the dataset
    available_columns = []
    missing_columns = []
    
    for col in correlation_columns:
        if col in combined_df.columns:
            available_columns.append(col)
        else:
            missing_columns.append(col)
    
    print(f"📊 Available columns: {len(available_columns)}/{len(correlation_columns)}")
    if missing_columns:
        print(f"⚠️  Missing columns: {missing_columns}")
    
    # Prepare data for correlation analysis
    correlation_data = combined_df[available_columns].copy()
    
    # Convert boolean columns to numeric
    bool_columns = ['original_answer_correctness', 'verifier_answer_correctness']
    for col in bool_columns:
        if col in correlation_data.columns:
            correlation_data[col] = correlation_data[col].astype(int)
    
    # Calculate correlation matrix
    correlation_matrix = correlation_data.corr()
    
    # Create the heatmap visualization
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    fig.suptitle('🔥 Comprehensive Correlation Analysis (14x14)', fontsize=18, fontweight='bold')
    
    # 1. Main correlation heatmap (top left)
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
    sns.heatmap(correlation_matrix, 
                annot=True, 
                fmt='.3f',
                cmap='RdBu_r',
                center=0,
                square=True,
                mask=mask,
                cbar_kws={'label': 'Correlation Coefficient'},
                ax=axes[0,0])
    axes[0,0].set_title('Correlation Heatmap (Lower Triangle)', fontweight='bold')
    axes[0,0].tick_params(axis='x', rotation=45, labelsize=9)
    axes[0,0].tick_params(axis='y', rotation=0, labelsize=9)
    
    # 2. Full correlation heatmap without mask (top right)
    sns.heatmap(correlation_matrix,
                annot=False,  # No annotations for better visibility
                cmap='RdBu_r',
                center=0,
                square=True,
                cbar_kws={'label': 'Correlation Coefficient'},
                ax=axes[0,1])
    axes[0,1].set_title('Full Correlation Heatmap', fontweight='bold')
    axes[0,1].tick_params(axis='x', rotation=45, labelsize=9)
    axes[0,1].tick_params(axis='y', rotation=0, labelsize=9)
    
    # 3. Correlation strength distribution (bottom left)
    correlation_values = correlation_matrix.values
    correlation_flat = correlation_values[np.triu_indices_from(correlation_values, k=1)]  # Upper triangle excluding diagonal
    
    axes[1,0].hist(correlation_flat, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1,0].axvline(0, color='red', linestyle='--', alpha=0.7, label='Zero correlation')
    axes[1,0].axvline(np.mean(correlation_flat), color='green', linestyle='--', alpha=0.7, 
                     label=f'Mean: {np.mean(correlation_flat):.3f}')
    axes[1,0].set_xlabel('Correlation Coefficient')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Distribution of Correlation Values')
    axes[1,0].legend()
    axes[1,0].grid(axis='y', alpha=0.3)
    
    # 4. Top correlations analysis (bottom right)
    axes[1,1].axis('off')
    
    # Find strongest positive and negative correlations
    correlation_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if not np.isnan(corr_value):
                correlation_pairs.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j], 
                    corr_value
                ))
    
    # Sort by absolute correlation value
    correlation_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
    
    # Display top correlations
    top_correlations_text = "🔝 STRONGEST CORRELATIONS:\n" + "="*40 + "\n\n"
    
    top_correlations_text += "📈 STRONGEST POSITIVE:\n"
    positive_corrs = [pair for pair in correlation_pairs if pair[2] > 0][:5]
    for i, (col1, col2, corr) in enumerate(positive_corrs, 1):
        col1_short = col1.replace('_', ' ').title()[:20]
        col2_short = col2.replace('_', ' ').title()[:20]
        top_correlations_text += f"{i}. {col1_short} ↔ {col2_short}\n   r = {corr:.3f}\n\n"
    
    top_correlations_text += "📉 STRONGEST NEGATIVE:\n"
    negative_corrs = [pair for pair in correlation_pairs if pair[2] < 0][:5]
    for i, (col1, col2, corr) in enumerate(negative_corrs, 1):
        col1_short = col1.replace('_', ' ').title()[:20]
        col2_short = col2.replace('_', ' ').title()[:20]
        top_correlations_text += f"{i}. {col1_short} ↔ {col2_short}\n   r = {corr:.3f}\n\n"
    
    # Calculate correlation strength categories
    strong_pos = len([pair for pair in correlation_pairs if pair[2] >= 0.7])
    moderate_pos = len([pair for pair in correlation_pairs if 0.3 <= pair[2] < 0.7])
    weak_pos = len([pair for pair in correlation_pairs if 0.1 <= pair[2] < 0.3])
    negligible = len([pair for pair in correlation_pairs if -0.1 < pair[2] < 0.1])
    weak_neg = len([pair for pair in correlation_pairs if -0.3 < pair[2] <= -0.1])
    moderate_neg = len([pair for pair in correlation_pairs if -0.7 < pair[2] <= -0.3])
    strong_neg = len([pair for pair in correlation_pairs if pair[2] <= -0.7])
    
    top_correlations_text += f"\n📊 CORRELATION STRENGTH SUMMARY:\n"
    top_correlations_text += f"Strong Positive (≥0.7): {strong_pos}\n"
    top_correlations_text += f"Moderate Positive (0.3-0.7): {moderate_pos}\n"
    top_correlations_text += f"Weak Positive (0.1-0.3): {weak_pos}\n"
    top_correlations_text += f"Negligible (-0.1 to 0.1): {negligible}\n"
    top_correlations_text += f"Weak Negative (-0.3 to -0.1): {weak_neg}\n"
    top_correlations_text += f"Moderate Negative (-0.7 to -0.3): {moderate_neg}\n"
    top_correlations_text += f"Strong Negative (≤-0.7): {strong_neg}\n"
    
    axes[1,1].text(0.05, 0.95, top_correlations_text, transform=axes[1,1].transAxes, 
                   fontsize=10, verticalalignment='top', fontfamily='monospace',
                   bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.3))
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed correlation analysis
    print(f"\n📋 DETAILED CORRELATION ANALYSIS:")
    print("="*50)
    
    print(f"📊 Dataset Statistics:")
    print(f"   • Total correlations calculated: {len(correlation_pairs)}")
    print(f"   • Mean correlation: {np.mean([pair[2] for pair in correlation_pairs]):.3f}")
    print(f"   • Std correlation: {np.std([pair[2] for pair in correlation_pairs]):.3f}")
    print(f"   • Max correlation: {max([pair[2] for pair in correlation_pairs]):.3f}")
    print(f"   • Min correlation: {min([pair[2] for pair in correlation_pairs]):.3f}")
    
    # Version-specific correlation analysis
    print(f"\n🔍 VERSION-SPECIFIC CORRELATION INSIGHTS:")
    for version in combined_df['version'].unique():
        version_data = combined_df[combined_df['version'] == version]
        version_corr_data = version_data[available_columns].copy()
        
        # Convert boolean columns to numeric for this version
        for col in bool_columns:
            if col in version_corr_data.columns:
                version_corr_data[col] = version_corr_data[col].astype(int)
        
        if len(version_corr_data) > 2:  # Need sufficient data for correlation
            version_corr_matrix = version_corr_data.corr()
            
            # Find strongest correlation for this version
            version_pairs = []
            for i in range(len(version_corr_matrix.columns)):
                for j in range(i+1, len(version_corr_matrix.columns)):
                    corr_value = version_corr_matrix.iloc[i, j]
                    if not np.isnan(corr_value):
                        version_pairs.append((
                            version_corr_matrix.columns[i],
                            version_corr_matrix.columns[j], 
                            corr_value
                        ))
            
            if version_pairs:
                strongest_corr = max(version_pairs, key=lambda x: abs(x[2]))
                mean_corr = np.mean([abs(pair[2]) for pair in version_pairs])
                print(f"   • {version}: Strongest |r|={abs(strongest_corr[2]):.3f}, Mean |r|={mean_corr:.3f}")
    
    return {
        'correlation_matrix': correlation_matrix,
        'correlation_pairs': correlation_pairs,
        'available_columns': available_columns,
        'missing_columns': missing_columns,
        'correlation_stats': {
            'mean': np.mean([pair[2] for pair in correlation_pairs]),
            'std': np.std([pair[2] for pair in correlation_pairs]),
            'max': max([pair[2] for pair in correlation_pairs]) if correlation_pairs else 0,
            'min': min([pair[2] for pair in correlation_pairs]) if correlation_pairs else 0
        }
    }

# Run correlation heatmap analysis
correlation_results = create_correlation_heatmap_analysis(combined_df)


In [ ]:
def generate_steps_llm_relationship_report(steps_llm_results, combined_df, save_to_file=False, filename='steps_llm_relationship_report.txt'):
    """
    Generate a comprehensive text report of steps vs LLM calls relationship analysis
    """
    
    report = []
    report.append("=" * 80)
    report.append("ORIGINAL STEPS vs LLM CALLS RELATIONSHIP ANALYSIS REPORT")
    report.append("Resource Scaling and Predictability Analysis Across Verification Versions")
    report.append("=" * 80)
    report.append("")
    
    # Executive Summary
    report.append("EXECUTIVE SUMMARY")
    report.append("-" * 50)
    
    version_correlations = steps_llm_results['version_correlations']
    mean_correlation = steps_llm_results['mean_correlation']
    correlation_range = steps_llm_results['correlation_range']
    
    report.append(f"Versions analyzed: {len(version_correlations)}")
    report.append(f"Overall relationship strength: {mean_correlation:.3f}")
    report.append(f"Correlation range: {correlation_range[0]:.3f} to {correlation_range[1]:.3f}")
    report.append("")
    
    # Interpret overall relationship strength
    if mean_correlation >= 0.7:
        relationship_strength = "STRONG"
        predictability = "Highly predictable"
        implication = "LLM usage can be reliably estimated from problem complexity"
    elif mean_correlation >= 0.5:
        relationship_strength = "MODERATE"
        predictability = "Moderately predictable"
        implication = "LLM usage somewhat predictable from problem complexity"
    elif mean_correlation >= 0.3:
        relationship_strength = "WEAK"
        predictability = "Weakly predictable"
        implication = "Limited predictability of LLM usage from step count"
    else:
        relationship_strength = "MINIMAL"
        predictability = "Unpredictable"
        implication = "LLM usage not well predicted by step count alone"
    
    report.append(f"Overall Relationship: {relationship_strength}")
    report.append(f"Predictability: {predictability}")
    report.append(f"Practical Implication: {implication}")
    report.append("")
    
    # Individual version analysis
    report.append("INDIVIDUAL VERSION ANALYSIS")
    report.append("-" * 50)
    
    # Sort versions by correlation strength
    sorted_versions = sorted(version_correlations.items(), key=lambda x: abs(x[1]), reverse=True)
    
    for rank, (version, correlation) in enumerate(sorted_versions, 1):
        version_data = combined_df[combined_df['version'] == version]
        valid_data = version_data.dropna(subset=['original_total_steps', 'total_llm_calls'])
        
        if len(valid_data) > 1:
            # Calculate additional statistics
            steps_mean = valid_data['original_total_steps'].mean()
            steps_std = valid_data['original_total_steps'].std()
            llm_mean = valid_data['total_llm_calls'].mean()
            llm_std = valid_data['total_llm_calls'].std()
            
            # Calculate linear regression parameters
            slope, intercept = np.polyfit(valid_data['original_total_steps'], valid_data['total_llm_calls'], 1)
            
            # Interpret correlation strength for this version
            if abs(correlation) >= 0.7:
                strength = "Strong"
                interpretation = "Highly predictable scaling"
            elif abs(correlation) >= 0.5:
                strength = "Moderate"
                interpretation = "Moderately predictable scaling"
            elif abs(correlation) >= 0.3:
                strength = "Weak"
                interpretation = "Limited predictability"
            else:
                strength = "Minimal"
                interpretation = "Unpredictable scaling"
            
            report.append(f"{rank}. {version}")
            report.append(f"   Correlation coefficient (r): {correlation:6.3f}")
            report.append(f"   Relationship strength: {strength}")
            report.append(f"   Interpretation: {interpretation}")
            report.append(f"   Sample size: {len(valid_data)} problems")
            report.append("")
            report.append(f"   Descriptive Statistics:")
            report.append(f"     Original Steps: Mean = {steps_mean:5.1f}, SD = {steps_std:5.1f}")
            report.append(f"     LLM Calls:      Mean = {llm_mean:5.1f}, SD = {llm_std:5.1f}")
            report.append("")
            report.append(f"   Linear Relationship:")
            report.append(f"     LLM Calls = {slope:.3f} × Original Steps + {intercept:.3f}")
            report.append(f"     Slope interpretation: {slope:.3f} additional LLM calls per step")
            if intercept > 0:
                report.append(f"     Baseline: {intercept:.1f} LLM calls (fixed overhead)")
            else:
                report.append(f"     Baseline: {abs(intercept):.1f} LLM calls saved (efficiency)")
            report.append("")
            
            # Calculate R-squared
            r_squared = correlation ** 2
            report.append(f"   Explained Variance (R²): {r_squared:.3f} ({r_squared*100:.1f}%)")
            report.append(f"   → {r_squared*100:.1f}% of LLM call variation explained by step count")
            report.append("")
            
            # Practical implications for this version
            if abs(correlation) >= 0.5:
                report.append(f"   Resource Planning Capability: HIGH")
                report.append(f"   → Can estimate LLM usage with {r_squared*100:.0f}% accuracy")
                report.append(f"   → Suitable for cost prediction and capacity planning")
            elif abs(correlation) >= 0.3:
                report.append(f"   Resource Planning Capability: MODERATE")
                report.append(f"   → Some predictability but other factors also important")
                report.append(f"   → Use with caution for resource estimation")
            else:
                report.append(f"   Resource Planning Capability: LOW")
                report.append(f"   → Poor predictability from step count alone")
                report.append(f"   → Consider additional complexity metrics")
            report.append("")
            report.append("-" * 40)
            report.append("")
    
    # Cross-version comparison
    report.append("CROSS-VERSION COMPARISON")
    report.append("-" * 50)
    
    best_version = max(version_correlations.items(), key=lambda x: abs(x[1]))
    worst_version = min(version_correlations.items(), key=lambda x: abs(x[1]))
    
    report.append(f"Most Predictable Version: {best_version[0]}")
    report.append(f"  Correlation: {best_version[1]:.3f}")
    report.append(f"  Resource planning: Highly reliable")
    report.append("")
    report.append(f"Least Predictable Version: {worst_version[0]}")
    report.append(f"  Correlation: {worst_version[1]:.3f}")
    report.append(f"  Resource planning: Unreliable")
    report.append("")
    
    # Calculate coefficient of variation across versions
    correlations_list = list(version_correlations.values())
    cv = np.std(correlations_list) / np.mean(np.abs(correlations_list)) if np.mean(np.abs(correlations_list)) > 0 else 0
    
    report.append(f"Version Consistency Analysis:")
    report.append(f"  Correlation range: {correlation_range[0]:.3f} to {correlation_range[1]:.3f}")
    report.append(f"  Coefficient of variation: {cv:.3f}")
    
    if cv < 0.2:
        report.append(f"  → HIGH CONSISTENCY: All versions show similar scaling patterns")
    elif cv < 0.5:
        report.append(f"  → MODERATE CONSISTENCY: Some variation in scaling patterns")
    else:
        report.append(f"  → LOW CONSISTENCY: Significant differences in scaling patterns")
    report.append("")
    
    # Efficiency analysis
    report.append("EFFICIENCY SCALING ANALYSIS")
    report.append("-" * 50)
    
    for version, correlation in sorted_versions:
        version_data = combined_df[combined_df['version'] == version]
        valid_data = version_data.dropna(subset=['original_total_steps', 'total_llm_calls'])
        
        if len(valid_data) > 1:
            slope, intercept = np.polyfit(valid_data['original_total_steps'], valid_data['total_llm_calls'], 1)
            
            # Efficiency interpretation
            if slope < 1.0:
                efficiency_rating = "HIGHLY EFFICIENT"
                efficiency_desc = f"Uses <1 LLM call per step on average"
            elif slope < 1.5:
                efficiency_rating = "EFFICIENT"
                efficiency_desc = f"Uses ~1-1.5 LLM calls per step"
            elif slope < 2.0:
                efficiency_rating = "MODERATE"
                efficiency_desc = f"Uses ~1.5-2 LLM calls per step"
            else:
                efficiency_rating = "INEFFICIENT"
                efficiency_desc = f"Uses >2 LLM calls per step"
            
            report.append(f"{version}:")
            report.append(f"  Scaling rate: {slope:.3f} LLM calls per step")
            report.append(f"  Efficiency: {efficiency_rating}")
            report.append(f"  Description: {efficiency_desc}")
            
            # Calculate cost implications (assuming cost per LLM call)
            if intercept > 0:
                report.append(f"  Fixed overhead: {intercept:.1f} LLM calls per problem")
            else:
                report.append(f"  Efficiency bonus: {abs(intercept):.1f} LLM calls saved")
            report.append("")
    
    # Predictive modeling insights
    report.append("PREDICTIVE MODELING INSIGHTS")
    report.append("-" * 50)
    
    high_predictability_versions = [v for v, r in version_correlations.items() if abs(r) >= 0.5]
    moderate_predictability_versions = [v for v, r in version_correlations.items() if 0.3 <= abs(r) < 0.5]
    low_predictability_versions = [v for v, r in version_correlations.items() if abs(r) < 0.3]
    
    report.append(f"High Predictability Versions ({len(high_predictability_versions)}):")
    for version in high_predictability_versions:
        corr = version_correlations[version]
        report.append(f"  • {version}: r = {corr:.3f} (R² = {corr**2:.3f})")
    report.append("")
    
    report.append(f"Moderate Predictability Versions ({len(moderate_predictability_versions)}):")
    for version in moderate_predictability_versions:
        corr = version_correlations[version]
        report.append(f"  • {version}: r = {corr:.3f} (R² = {corr**2:.3f})")
    report.append("")
    
    report.append(f"Low Predictability Versions ({len(low_predictability_versions)}):")
    for version in low_predictability_versions:
        corr = version_correlations[version]
        report.append(f"  • {version}: r = {corr:.3f} (R² = {corr**2:.3f})")
    report.append("")
    
    # Practical recommendations
    report.append("PRACTICAL RECOMMENDATIONS")
    report.append("-" * 50)
    
    report.append("Resource Planning:")
    if len(high_predictability_versions) > 0:
        report.append(f"  • Use {', '.join(high_predictability_versions)} for reliable cost estimation")
        report.append(f"  • Step count provides good proxy for LLM usage")
        report.append(f"  • Implement step-based capacity planning")
    else:
        report.append(f"  • Step count alone insufficient for resource planning")
        report.append(f"  • Consider additional complexity metrics")
        report.append(f"  • Implement dynamic resource allocation")
    report.append("")
    
    report.append("Version Selection:")
    if best_version[1] >= 0.5:
        report.append(f"  • Recommended: {best_version[0]} (most predictable scaling)")
        report.append(f"  • Provides {(best_version[1]**2)*100:.0f}% resource predictability")
    report.append(f"  • Avoid: {worst_version[0]} (unpredictable scaling)")
    report.append("")
    
    report.append("Cost Optimization:")
    # Find most efficient version (lowest slope)
    efficiency_ranking = []
    for version, correlation in version_correlations.items():
        version_data = combined_df[combined_df['version'] == version]
        valid_data = version_data.dropna(subset=['original_total_steps', 'total_llm_calls'])
        if len(valid_data) > 1:
            slope, _ = np.polyfit(valid_data['original_total_steps'], valid_data['total_llm_calls'], 1)
            efficiency_ranking.append((version, slope))
    
    if efficiency_ranking:
        most_efficient = min(efficiency_ranking, key=lambda x: x[1])
        report.append(f"  • Most efficient: {most_efficient[0]} ({most_efficient[1]:.3f} LLM calls per step)")
        report.append(f"  • Focus optimization efforts on inefficient versions")
        report.append(f"  • Monitor scaling rates for cost control")
    
    report.append("")
    report.append("Quality Assurance:")
    report.append(f"  • Set up monitoring for step-to-LLM call ratios")
    report.append(f"  • Alert on deviations from expected scaling patterns")
    report.append(f"  • Regular recalibration of prediction models")
    report.append("")
    
    report.append("=" * 80)
    report.append("END OF STEPS vs LLM CALLS RELATIONSHIP ANALYSIS REPORT")
    report.append("=" * 80)
    
    # Convert to string
    report_text = "\n".join(report)
    
    # Print to console
    print(report_text)
    
    # Save to file if requested
    if save_to_file:
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(report_text)
            print(f"\n✅ Steps vs LLM calls report saved to {filename}")
        except Exception as e:
            print(f"\n❌ Error saving steps vs LLM calls report: {e}")
    
    return report_text

# Run steps vs LLM calls analysis
steps_llm_results = generate_steps_llm_relationship_report(combined_df)

## CORRELATION HEATMAP TEXT REPORT GENERATOR

In [ ]:
def generate_correlation_heatmap_report(correlation_results, combined_df, save_to_file=False, filename='correlation_analysis_report.txt'):
    """
    Generate a comprehensive text report of correlation heatmap analysis
    """
    
    report = []
    report.append("=" * 80)
    report.append("CORRELATION HEATMAP ANALYSIS REPORT")
    report.append("Multi-Variable Relationship Analysis Across Verification System Metrics")
    report.append("=" * 80)
    report.append("")
    
    # Executive Summary
    report.append("EXECUTIVE SUMMARY")
    report.append("-" * 50)
    
    correlation_matrix = correlation_results['correlation_matrix']
    correlation_pairs = correlation_results['correlation_pairs']
    available_columns = correlation_results['available_columns']
    missing_columns = correlation_results['missing_columns']
    stats = correlation_results['correlation_stats']
    
    report.append(f"Total variables analyzed: {len(available_columns)}")
    report.append(f"Total correlation pairs: {len(correlation_pairs)}")
    report.append(f"Missing variables: {len(missing_columns)}")
    if missing_columns:
        report.append(f"  Missing: {', '.join(missing_columns)}")
    report.append("")
    
    report.append("Correlation Statistics:")
    report.append(f"  Mean correlation: {stats['mean']:.3f}")
    report.append(f"  Standard deviation: {stats['std']:.3f}")
    report.append(f"  Range: {stats['min']:.3f} to {stats['max']:.3f}")
    report.append("")
    
    # Correlation strength distribution
    strong_pos = len([pair for pair in correlation_pairs if pair[2] >= 0.7])
    moderate_pos = len([pair for pair in correlation_pairs if 0.3 <= pair[2] < 0.7])
    weak_pos = len([pair for pair in correlation_pairs if 0.1 <= pair[2] < 0.3])
    negligible = len([pair for pair in correlation_pairs if -0.1 < pair[2] < 0.1])
    weak_neg = len([pair for pair in correlation_pairs if -0.3 < pair[2] <= -0.1])
    moderate_neg = len([pair for pair in correlation_pairs if -0.7 < pair[2] <= -0.3])
    strong_neg = len([pair for pair in correlation_pairs if pair[2] <= -0.7])
    
    total_pairs = len(correlation_pairs)
    
    report.append("Correlation Strength Distribution:")
    report.append(f"  Strong Positive (≥0.7):     {strong_pos:3d} ({(strong_pos/total_pairs)*100:5.1f}%)")
    report.append(f"  Moderate Positive (0.3-0.7): {moderate_pos:3d} ({(moderate_pos/total_pairs)*100:5.1f}%)")
    report.append(f"  Weak Positive (0.1-0.3):     {weak_pos:3d} ({(weak_pos/total_pairs)*100:5.1f}%)")
    report.append(f"  Negligible (-0.1 to 0.1):    {negligible:3d} ({(negligible/total_pairs)*100:5.1f}%)")
    report.append(f"  Weak Negative (-0.3 to -0.1): {weak_neg:3d} ({(weak_neg/total_pairs)*100:5.1f}%)")
    report.append(f"  Moderate Negative (-0.7 to -0.3): {moderate_neg:3d} ({(moderate_neg/total_pairs)*100:5.1f}%)")
    report.append(f"  Strong Negative (≤-0.7):     {strong_neg:3d} ({(strong_neg/total_pairs)*100:5.1f}%)")
    report.append("")
    
    # Top correlations
    report.append("STRONGEST CORRELATIONS")
    report.append("-" * 50)
    
    # Sort by absolute correlation value
    sorted_pairs = sorted(correlation_pairs, key=lambda x: abs(x[2]), reverse=True)
    
    report.append("Top 10 Strongest Positive Correlations:")
    positive_corrs = [pair for pair in sorted_pairs if pair[2] > 0][:10]
    for i, (col1, col2, corr) in enumerate(positive_corrs, 1):
        report.append(f"  {i:2d}. {col1:<25} ↔ {col2:<25} r = {corr:6.3f}")
    report.append("")
    
    report.append("Top 10 Strongest Negative Correlations:")
    negative_corrs = [pair for pair in sorted_pairs if pair[2] < 0][:10]
    for i, (col1, col2, corr) in enumerate(negative_corrs, 1):
        report.append(f"  {i:2d}. {col1:<25} ↔ {col2:<25} r = {corr:6.3f}")
    report.append("")
    
    # Complete correlation matrix
    report.append("COMPLETE CORRELATION MATRIX")
    report.append("-" * 50)
    
    # Create formatted correlation matrix table
    matrix_df = correlation_matrix.round(3)
    
    # Header
    col_names = list(matrix_df.columns)
    header = "Variable".ljust(25)
    for i, col in enumerate(col_names):
        header += f"{i+1:>8}"
    report.append(header)
    report.append("-" * len(header))
    
    # Matrix rows
    for i, (index, row) in enumerate(matrix_df.iterrows()):
        row_str = f"{i+1:2d}. {index[:20]:<20}"
        for j, val in enumerate(row):
            if i == j:
                row_str += f"{'1.000':>8}"
            elif i > j:
                row_str += f"{val:8.3f}"
            else:
                row_str += f"{'':>8}"  # Empty for upper triangle
        report.append(row_str)
    
    report.append("")
    report.append("Variable Legend:")
    for i, col in enumerate(col_names):
        report.append(f"  {i+1:2d}. {col}")
    report.append("")
    
    # Key relationship analysis
    report.append("KEY RELATIONSHIP ANALYSIS")
    report.append("-" * 50)
    
    # Performance relationships
    performance_vars = ['original_answer_correctness', 'verifier_answer_correctness']
    efficiency_vars = ['processing_time_ms', 'total_llm_calls', 'total_input_tokens', 'total_output_tokens']
    quality_vars = ['original_total_steps', 'verified_total_steps']
    
    report.append("Performance Correlations:")
    for perf_var in performance_vars:
        if perf_var in correlation_matrix.columns:
            report.append(f"  {perf_var}:")
            perf_corrs = correlation_matrix[perf_var].drop(perf_var).sort_values(key=abs, ascending=False)
            for var, corr in perf_corrs.head(5).items():
                if abs(corr) > 0.1:  # Only show meaningful correlations
                    report.append(f"    → {var:<25}: {corr:6.3f}")
    report.append("")
    
    report.append("Efficiency Correlations:")
    for eff_var in efficiency_vars:
        if eff_var in correlation_matrix.columns:
            report.append(f"  {eff_var}:")
            eff_corrs = correlation_matrix[eff_var].drop(eff_var).sort_values(key=abs, ascending=False)
            for var, corr in eff_corrs.head(3).items():
                if abs(corr) > 0.1:
                    report.append(f"    → {var:<25}: {corr:6.3f}")
    report.append("")
    
    # Version-specific analysis
    report.append("VERSION-SPECIFIC CORRELATION PATTERNS")
    report.append("-" * 50)
    
    versions = combined_df['version'].unique()
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        version_corr_data = version_data[available_columns].copy()
        
        # Convert boolean columns to numeric
        bool_columns = ['original_answer_correctness', 'verifier_answer_correctness']
        for col in bool_columns:
            if col in version_corr_data.columns:
                version_corr_data[col] = version_corr_data[col].astype(int)
        
        if len(version_corr_data) > 2:
            version_corr_matrix = version_corr_data.corr()
            
            # Find strongest correlations for this version
            version_pairs = []
            for i in range(len(version_corr_matrix.columns)):
                for j in range(i+1, len(version_corr_matrix.columns)):
                    corr_value = version_corr_matrix.iloc[i, j]
                    if not np.isnan(corr_value) and abs(corr_value) > 0.3:  # Only meaningful correlations
                        version_pairs.append((
                            version_corr_matrix.columns[i],
                            version_corr_matrix.columns[j], 
                            corr_value
                        ))
            
            version_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
            
            report.append(f"{version}:")
            report.append(f"  Sample size: {len(version_data)}")
            if version_pairs:
                report.append(f"  Top correlations (|r| > 0.3):")
                for col1, col2, corr in version_pairs[:5]:
                    report.append(f"    {col1:<20} ↔ {col2:<20}: {corr:6.3f}")
            else:
                report.append(f"  No strong correlations found (|r| > 0.3)")
            
            # Calculate average correlation strength
            all_corrs = [abs(pair[2]) for pair in version_pairs]
            if all_corrs:
                avg_strength = np.mean(all_corrs)
                report.append(f"  Average correlation strength: {avg_strength:.3f}")
            report.append("")
    
    # Statistical insights
    report.append("STATISTICAL INSIGHTS")
    report.append("-" * 50)
    
    if strong_pos + strong_neg > total_pairs * 0.1:
        report.append("🔍 HIGH MULTICOLLINEARITY DETECTED")
        report.append("  Multiple strong correlations (|r| ≥ 0.7) found.")
        report.append("  Consider dimensionality reduction or variable selection.")
    elif moderate_pos + moderate_neg > total_pairs * 0.3:
        report.append("📊 MODERATE CORRELATION STRUCTURE")
        report.append("  Moderate correlations suggest meaningful relationships.")
        report.append("  Variables show expected interdependencies.")
    else:
        report.append("✅ LOW CORRELATION STRUCTURE")
        report.append("  Most variables are relatively independent.")
        report.append("  Good for regression modeling and analysis.")
    report.append("")
    
    # Practical implications
    report.append("PRACTICAL IMPLICATIONS")
    report.append("-" * 50)
    
    report.append("Model Building Considerations:")
    if strong_pos + strong_neg > 5:
        report.append("  • High multicollinearity detected - consider feature selection")
        report.append("  • Use regularization techniques (Ridge/Lasso) for regression")
        report.append("  • Consider principal component analysis for dimensionality reduction")
    else:
        report.append("  • Low multicollinearity - most variables suitable for modeling")
        report.append("  • Standard regression techniques applicable")
    report.append("")
    
    report.append("System Optimization Insights:")
    # Find key efficiency relationships
    efficiency_relationships = []
    for pair in correlation_pairs:
        col1, col2, corr = pair
        if any(eff in col1 for eff in ['time', 'token', 'llm']) and any(eff in col2 for eff in ['time', 'token', 'llm']):
            if abs(corr) > 0.5:
                efficiency_relationships.append(pair)
    
    if efficiency_relationships:
        report.append("  Key efficiency relationships identified:")
        for col1, col2, corr in efficiency_relationships[:3]:
            if corr > 0:
                report.append(f"    • {col1} and {col2} increase together (r={corr:.3f})")
            else:
                report.append(f"    • {col1} increases as {col2} decreases (r={corr:.3f})")
    
    report.append("")
    report.append("Quality Monitoring Recommendations:")
    report.append("  • Monitor strongly correlated variables together")
    report.append("  • Set up alerts for unusual correlation pattern changes")
    report.append("  • Use correlation patterns for anomaly detection")
    report.append("")
    
    report.append("=" * 80)
    report.append("END OF CORRELATION HEATMAP ANALYSIS REPORT")
    report.append("=" * 80)
    
    # Convert to string
    report_text = "\n".join(report)
    
    # Print to console
    print(report_text)
    
    # Save to file if requested
    if save_to_file:
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(report_text)
            print(f"\n✅ Correlation report saved to {filename}")
        except Exception as e:
            print(f"\n❌ Error saving correlation report: {e}")
    
    return report_text


## STEPS VS LLM CALLS RELATIONSHIP TEXT REPORT GENERATOR

In [ ]:
def generate_steps_llm_relationship_report(steps_llm_results, combined_df, save_to_file=False, filename='steps_llm_relationship_report.txt'):
    """
    Generate a comprehensive text report of steps vs LLM calls relationship analysis
    """
    
    report = []
    report.append("=" * 80)
    report.append("ORIGINAL STEPS vs LLM CALLS RELATIONSHIP ANALYSIS REPORT")
    report.append("Resource Scaling and Predictability Analysis Across Verification Versions")
    report.append("=" * 80)
    report.append("")
    
    # Executive Summary
    report.append("EXECUTIVE SUMMARY")
    report.append("-" * 50)
    
    version_correlations = steps_llm_results['version_correlations']
    mean_correlation = steps_llm_results['mean_correlation']
    correlation_range = steps_llm_results['correlation_range']
    
    report.append(f"Versions analyzed: {len(version_correlations)}")
    report.append(f"Overall relationship strength: {mean_correlation:.3f}")
    report.append(f"Correlation range: {correlation_range[0]:.3f} to {correlation_range[1]:.3f}")
    report.append("")
    
    # Interpret overall relationship strength
    if mean_correlation >= 0.7:
        relationship_strength = "STRONG"
        predictability = "Highly predictable"
        implication = "LLM usage can be reliably estimated from problem complexity"
    elif mean_correlation >= 0.5:
        relationship_strength = "MODERATE"
        predictability = "Moderately predictable"
        implication = "LLM usage somewhat predictable from problem complexity"
    elif mean_correlation >= 0.3:
        relationship_strength = "WEAK"
        predictability = "Weakly predictable"
        implication = "Limited predictability of LLM usage from step count"
    else:
        relationship_strength = "MINIMAL"
        predictability = "Unpredictable"
        implication = "LLM usage not well predicted by step count alone"
    
    report.append(f"Overall Relationship: {relationship_strength}")
    report.append(f"Predictability: {predictability}")
    report.append(f"Practical Implication: {implication}")
    report.append("")
    
    # Individual version analysis
    report.append("INDIVIDUAL VERSION ANALYSIS")
    report.append("-" * 50)
    
    # Sort versions by correlation strength
    sorted_versions = sorted(version_correlations.items(), key=lambda x: abs(x[1]), reverse=True)
    
    for rank, (version, correlation) in enumerate(sorted_versions, 1):
        version_data = combined_df[combined_df['version'] == version]
        valid_data = version_data.dropna(subset=['original_total_steps', 'total_llm_calls'])
        
        if len(valid_data) > 1:
            # Calculate additional statistics
            steps_mean = valid_data['original_total_steps'].mean()
            steps_std = valid_data['original_total_steps'].std()
            llm_mean = valid_data['total_llm_calls'].mean()
            llm_std = valid_data['total_llm_calls'].std()
            
            # Calculate linear regression parameters
            slope, intercept = np.polyfit(valid_data['original_total_steps'], valid_data['total_llm_calls'], 1)
            
            # Interpret correlation strength for this version
            if abs(correlation) >= 0.7:
                strength = "Strong"
                interpretation = "Highly predictable scaling"
            elif abs(correlation) >= 0.5:
                strength = "Moderate"
                interpretation = "Moderately predictable scaling"
            elif abs(correlation) >= 0.3:
                strength = "Weak"
                interpretation = "Limited predictability"
            else:
                strength = "Minimal"
                interpretation = "Unpredictable scaling"
            
            report.append(f"{rank}. {version}")
            report.append(f"   Correlation coefficient (r): {correlation:6.3f}")
            report.append(f"   Relationship strength: {strength}")
            report.append(f"   Interpretation: {interpretation}")
            report.append(f"   Sample size: {len(valid_data)} problems")
            report.append("")
            report.append(f"   Descriptive Statistics:")
            report.append(f"     Original Steps: Mean = {steps_mean:5.1f}, SD = {steps_std:5.1f}")
            report.append(f"     LLM Calls:      Mean = {llm_mean:5.1f}, SD = {llm_std:5.1f}")
            report.append("")
            report.append(f"   Linear Relationship:")
            report.append(f"     LLM Calls = {slope:.3f} × Original Steps + {intercept:.3f}")
            report.append(f"     Slope interpretation: {slope:.3f} additional LLM calls per step")
            if intercept > 0:
                report.append(f"     Baseline: {intercept:.1f} LLM calls (fixed overhead)")
            else:
                report.append(f"     Baseline: {abs(intercept):.1f} LLM calls saved (efficiency)")
            report.append("")
            
            # Calculate R-squared
            r_squared = correlation ** 2
            report.append(f"   Explained Variance (R²): {r_squared:.3f} ({r_squared*100:.1f}%)")
            report.append(f"   → {r_squared*100:.1f}% of LLM call variation explained by step count")
            report.append("")
            
            # Practical implications for this version
            if abs(correlation) >= 0.5:
                report.append(f"   Resource Planning Capability: HIGH")
                report.append(f"   → Can estimate LLM usage with {r_squared*100:.0f}% accuracy")
                report.append(f"   → Suitable for cost prediction and capacity planning")
            elif abs(correlation) >= 0.3:
                report.append(f"   Resource Planning Capability: MODERATE")
                report.append(f"   → Some predictability but other factors also important")
                report.append(f"   → Use with caution for resource estimation")
            else:
                report.append(f"   Resource Planning Capability: LOW")
                report.append(f"   → Poor predictability from step count alone")
                report.append(f"   → Consider additional complexity metrics")
            report.append("")
            report.append("-" * 40)
            report.append("")
    
    # Cross-version comparison
    report.append("CROSS-VERSION COMPARISON")
    report.append("-" * 50)
    
    best_version = max(version_correlations.items(), key=lambda x: abs(x[1]))
    worst_version = min(version_correlations.items(), key=lambda x: abs(x[1]))
    
    report.append(f"Most Predictable Version: {best_version[0]}")
    report.append(f"  Correlation: {best_version[1]:.3f}")
    report.append(f"  Resource planning: Highly reliable")
    report.append("")
    report.append(f"Least Predictable Version: {worst_version[0]}")
    report.append(f"  Correlation: {worst_version[1]:.3f}")
    report.append(f"  Resource planning: Unreliable")
    report.append("")
    
    # Calculate coefficient of variation across versions
    correlations_list = list(version_correlations.values())
    cv = np.std(correlations_list) / np.mean(np.abs(correlations_list)) if np.mean(np.abs(correlations_list)) > 0 else 0
    
    report.append(f"Version Consistency Analysis:")
    report.append(f"  Correlation range: {correlation_range[0]:.3f} to {correlation_range[1]:.3f}")
    report.append(f"  Coefficient of variation: {cv:.3f}")
    
    if cv < 0.2:
        report.append(f"  → HIGH CONSISTENCY: All versions show similar scaling patterns")
    elif cv < 0.5:
        report.append(f"  → MODERATE CONSISTENCY: Some variation in scaling patterns")
    else:
        report.append(f"  → LOW CONSISTENCY: Significant differences in scaling patterns")
    report.append("")
    
    # Efficiency analysis
    report.append("EFFICIENCY SCALING ANALYSIS")
    report.append("-" * 50)
    
    for version, correlation in sorted_versions:
        version_data = combined_df[combined_df['version'] == version]
        valid_data = version_data.dropna(subset=['original_total_steps', 'total_llm_calls'])
        
        if len(valid_data) > 1:
            slope, intercept = np.polyfit(valid_data['original_total_steps'], valid_data['total_llm_calls'], 1)
            
            # Efficiency interpretation
            if slope < 1.0:
                efficiency_rating = "HIGHLY EFFICIENT"
                efficiency_desc = f"Uses <1 LLM call per step on average"
            elif slope < 1.5:
                efficiency_rating = "EFFICIENT"
                efficiency_desc = f"Uses ~1-1.5 LLM calls per step"
            elif slope < 2.0:
                efficiency_rating = "MODERATE"
                efficiency_desc = f"Uses ~1.5-2 LLM calls per step"
            else:
                efficiency_rating = "INEFFICIENT"
                efficiency_desc = f"Uses >2 LLM calls per step"
            
            report.append(f"{version}:")
            report.append(f"  Scaling rate: {slope:.3f} LLM calls per step")
            report.append(f"  Efficiency: {efficiency_rating}")
            report.append(f"  Description: {efficiency_desc}")
            
            # Calculate cost implications (assuming cost per LLM call)
            if intercept > 0:
                report.append(f"  Fixed overhead: {intercept:.1f} LLM calls per problem")
            else:
                report.append(f"  Efficiency bonus: {abs(intercept):.1f} LLM calls saved")
            report.append("")
    
    # Predictive modeling insights
    report.append("PREDICTIVE MODELING INSIGHTS")
    report.append("-" * 50)
    
    high_predictability_versions = [v for v, r in version_correlations.items() if abs(r) >= 0.5]
    moderate_predictability_versions = [v for v, r in version_correlations.items() if 0.3 <= abs(r) < 0.5]
    low_predictability_versions = [v for v, r in version_correlations.items() if abs(r) < 0.3]
    
    report.append(f"High Predictability Versions ({len(high_predictability_versions)}):")
    for version in high_predictability_versions:
        corr = version_correlations[version]
        report.append(f"  • {version}: r = {corr:.3f} (R² = {corr**2:.3f})")
    report.append("")
    
    report.append(f"Moderate Predictability Versions ({len(moderate_predictability_versions)}):")
    for version in moderate_predictability_versions:
        corr = version_correlations[version]
        report.append(f"  • {version}: r = {corr:.3f} (R² = {corr**2:.3f})")
    report.append("")
    
    report.append(f"Low Predictability Versions ({len(low_predictability_versions)}):")
    for version in low_predictability_versions:
        corr = version_correlations[version]
        report.append(f"  • {version}: r = {corr:.3f} (R² = {corr**2:.3f})")
    report.append("")
    
    # Practical recommendations
    report.append("PRACTICAL RECOMMENDATIONS")
    report.append("-" * 50)
    
    report.append("Resource Planning:")
    if len(high_predictability_versions) > 0:
        report.append(f"  • Use {', '.join(high_predictability_versions)} for reliable cost estimation")
        report.append(f"  • Step count provides good proxy for LLM usage")
        report.append(f"  • Implement step-based capacity planning")
    else:
        report.append(f"  • Step count alone insufficient for resource planning")
        report.append(f"  • Consider additional complexity metrics")
        report.append(f"  • Implement dynamic resource allocation")
    report.append("")
    
    report.append("Version Selection:")
    if best_version[1] >= 0.5:
        report.append(f"  • Recommended: {best_version[0]} (most predictable scaling)")
        report.append(f"  • Provides {(best_version[1]**2)*100:.0f}% resource predictability")
    report.append(f"  • Avoid: {worst_version[0]} (unpredictable scaling)")
    report.append("")
    
    report.append("Cost Optimization:")
    # Find most efficient version (lowest slope)
    efficiency_ranking = []
    for version, correlation in version_correlations.items():
        version_data = combined_df[combined_df['version'] == version]
        valid_data = version_data.dropna(subset=['original_total_steps', 'total_llm_calls'])
        if len(valid_data) > 1:
            slope, _ = np.polyfit(valid_data['original_total_steps'], valid_data['total_llm_calls'], 1)
            efficiency_ranking.append((version, slope))
    
    if efficiency_ranking:
        most_efficient = min(efficiency_ranking, key=lambda x: x[1])
        report.append(f"  • Most efficient: {most_efficient[0]} ({most_efficient[1]:.3f} LLM calls per step)")
        report.append(f"  • Focus optimization efforts on inefficient versions")
        report.append(f"  • Monitor scaling rates for cost control")
    
    report.append("")
    report.append("Quality Assurance:")
    report.append(f"  • Set up monitoring for step-to-LLM call ratios")
    report.append(f"  • Alert on deviations from expected scaling patterns")
    report.append(f"  • Regular recalibration of prediction models")
    report.append("")
    
    report.append("=" * 80)
    report.append("END OF STEPS vs LLM CALLS RELATIONSHIP ANALYSIS REPORT")
    report.append("=" * 80)
    
    # Convert to string
    report_text = "\n".join(report)
    
    # Print to console
    print(report_text)
    
    # Save to file if requested
    if save_to_file:
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(report_text)
            print(f"\n✅ Steps vs LLM calls report saved to {filename}")
        except Exception as e:
            print(f"\n❌ Error saving steps vs LLM calls report: {e}")
    
    return report_text

## STUART-MAXWELL TEST ANALYSIS

In [ ]:
def create_stuart_maxwell_analysis(combined_df):
    """
    Perform Stuart-Maxwell test analysis on rating transition matrices for each version
    """
    print("\n" + "="*70)
    print("🧪 STUART-MAXWELL TEST ANALYSIS")
    print("="*70)
    
    # Define transition columns
    transition_columns = [
        'neg1_to_neg1', 'neg1_to_zero', 'neg1_to_pos1',
        'zero_to_neg1', 'zero_to_zero', 'zero_to_pos1', 
        'pos1_to_neg1', 'pos1_to_zero', 'pos1_to_pos1'
    ]
    
    # Check which transition columns exist
    available_transition_cols = [col for col in transition_columns if col in combined_df.columns]
    missing_transition_cols = [col for col in transition_columns if col not in combined_df.columns]
    
    print(f"📊 Available transition columns: {len(available_transition_cols)}/{len(transition_columns)}")
    if missing_transition_cols:
        print(f"⚠️  Missing columns: {missing_transition_cols}")
    
    if len(available_transition_cols) < 9:
        print(f"❌ Cannot perform complete Stuart-Maxwell analysis - need all 9 transition columns")
        return None
    
    versions = combined_df['version'].unique()
    stuart_maxwell_results = {}
    
    # Create visualization
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))
    fig.suptitle('🧪 Stuart-Maxwell Test Analysis: Rating Transition Matrices', fontsize=16, fontweight='bold')
    
    # Stuart-Maxwell test function
    def stuart_maxwell_test(transition_matrix):
        """
        Perform Stuart-Maxwell test for marginal homogeneity
        """
        try:
            from scipy.stats import chi2
            
            # Reshape 3x3 matrix to vectors
            matrix = np.array(transition_matrix).reshape(3, 3)
            
            # Calculate marginal totals
            row_marginals = matrix.sum(axis=1)  # Original ratings
            col_marginals = matrix.sum(axis=0)  # Verified ratings
            
            # Calculate differences
            d = row_marginals - col_marginals
            
            # Calculate variance-covariance matrix
            n = matrix.sum()
            if n == 0:
                return np.nan, np.nan, "No data"
            
            # Simplified Stuart-Maxwell statistic calculation
            # For 3x3 table, we use the first k-1=2 differences
            d_reduced = d[:-1]  # Remove last element (redundant)
            
            # Calculate covariance matrix elements
            V = np.zeros((2, 2))
            
            # Diagonal elements
            V[0, 0] = (row_marginals[0] + col_marginals[0] - 2*matrix[0, 0]) / n
            V[1, 1] = (row_marginals[1] + col_marginals[1] - 2*matrix[1, 1]) / n
            
            # Off-diagonal elements  
            V[0, 1] = V[1, 0] = -(matrix[0, 1] + matrix[1, 0]) / n
            
            # Check if V is invertible
            det_V = np.linalg.det(V)
            if abs(det_V) < 1e-10:
                return np.nan, np.nan, "Singular covariance matrix"
            
            # Calculate test statistic
            V_inv = np.linalg.inv(V)
            test_statistic = n * d_reduced.T @ V_inv @ d_reduced
            
            # Degrees of freedom (k-1 = 2 for 3x3 table)
            df = 2
            
            # Calculate p-value
            p_value = 1 - chi2.cdf(test_statistic, df)
            
            return test_statistic, p_value, "Success"
            
        except Exception as e:
            return np.nan, np.nan, f"Error: {str(e)}"
    
    # Analyze each version
    for idx, version in enumerate(versions):
        version_data = combined_df[combined_df['version'] == version]
        
        # Sum transition counts for this version
        transition_counts = version_data[available_transition_cols].sum()
        
        # Reshape to 3x3 matrix
        transition_matrix = transition_counts.values.reshape(3, 3)
        
        # Perform Stuart-Maxwell test
        test_stat, p_value, status = stuart_maxwell_test(transition_matrix)
        
        # Store results
        stuart_maxwell_results[version] = {
            'transition_matrix': transition_matrix,
            'test_statistic': test_stat,
            'p_value': p_value,
            'status': status,
            'sample_size': transition_matrix.sum(),
            'marginal_homogeneity': 'Rejected' if (not np.isnan(p_value) and p_value < 0.05) else 'Not Rejected'
        }
        
        # Create subplot for this version's transition matrix
        row = idx // 2
        col = idx % 2
        
        if row < 3:  # Only plot if we have space
            ax = axes[row, col]
            
            # Create heatmap
            im = ax.imshow(transition_matrix, cmap='Blues', aspect='auto')
            
            # Add text annotations
            for i in range(3):
                for j in range(3):
                    text = ax.text(j, i, int(transition_matrix[i, j]),
                                 ha="center", va="center", color="black", fontweight='bold')
            
            ax.set_xticks([0, 1, 2])
            ax.set_yticks([0, 1, 2])
            ax.set_xticklabels(['To -1', 'To 0', 'To +1'])
            ax.set_yticklabels(['From -1', 'From 0', 'From +1'])
            ax.set_title(f'{version}\nStuart-Maxwell: p={p_value:.4f}' if not np.isnan(p_value) else f'{version}\nTest Failed')
            
            # Add colorbar
            plt.colorbar(im, ax=ax, shrink=0.6)
    
    # Hide unused subplots
    for idx in range(len(versions), 6):
        row = idx // 2
        col = idx % 2
        if row < 3:
            axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed results
    print(f"\n📋 STUART-MAXWELL TEST RESULTS:")
    print("="*50)
    
    for version, results in stuart_maxwell_results.items():
        print(f"\n{version}:")
        print(f"  • Sample Size: {results['sample_size']}")
        print(f"  • Test Statistic: {results['test_statistic']:.4f}" if not np.isnan(results['test_statistic']) else "  • Test Statistic: Failed")
        print(f"  • P-value: {results['p_value']:.4f}" if not np.isnan(results['p_value']) else "  • P-value: N/A")
        print(f"  • Marginal Homogeneity: {results['marginal_homogeneity']}")
        print(f"  • Status: {results['status']}")
        
        # Calculate marginal totals for interpretation
        matrix = results['transition_matrix']
        row_marginals = matrix.sum(axis=1)
        col_marginals = matrix.sum(axis=0)
        
        print(f"  • Original Ratings: [-1: {row_marginals[0]}, 0: {row_marginals[1]}, +1: {row_marginals[2]}]")
        print(f"  • Verified Ratings: [-1: {col_marginals[0]}, 0: {col_marginals[1]}, +1: {col_marginals[2]}]")
    
    # Cross-version comparison analysis
    print(f"\n🔍 CROSS-VERSION COMPARISON:")
    print("="*40)
    
    # Compare test statistics
    valid_results = {v: r for v, r in stuart_maxwell_results.items() 
                    if not np.isnan(r['test_statistic'])}
    
    if len(valid_results) >= 2:
        # Find version with strongest evidence against marginal homogeneity
        max_test_stat = max(valid_results.values(), key=lambda x: x['test_statistic'])
        min_test_stat = min(valid_results.values(), key=lambda x: x['test_statistic'])
        
        max_version = [v for v, r in valid_results.items() if r['test_statistic'] == max_test_stat['test_statistic']][0]
        min_version = [v for v, r in valid_results.items() if r['test_statistic'] == min_test_stat['test_statistic']][0]
        
        print(f"🔥 Most asymmetric transitions: {max_version} (χ²={max_test_stat['test_statistic']:.3f})")
        print(f"✅ Most symmetric transitions: {min_version} (χ²={min_test_stat['test_statistic']:.3f})")
        
        # Count significant results
        significant_versions = [v for v, r in valid_results.items() 
                              if not np.isnan(r['p_value']) and r['p_value'] < 0.05]
        
        print(f"\n📊 SUMMARY STATISTICS:")
        print(f"  • Versions with significant asymmetry: {len(significant_versions)}/{len(valid_results)}")
        if significant_versions:
            print(f"  • Asymmetric versions: {', '.join(significant_versions)}")
        
        # Calculate improvement/degradation patterns
        print(f"\n📈 RATING CHANGE PATTERNS:")
        for version, results in stuart_maxwell_results.items():
            matrix = results['transition_matrix']
            total_transitions = matrix.sum()
            
            if total_transitions > 0:
                # Calculate improvement (moving to higher rating)
                improvements = matrix[0, 1] + matrix[0, 2] + matrix[1, 2]  # -1→0, -1→+1, 0→+1
                degradations = matrix[1, 0] + matrix[2, 0] + matrix[2, 1]  # 0→-1, +1→-1, +1→0
                unchanged = matrix[0, 0] + matrix[1, 1] + matrix[2, 2]     # -1→-1, 0→0, +1→+1
                
                improvement_rate = (improvements / total_transitions) * 100
                degradation_rate = (degradations / total_transitions) * 100
                stability_rate = (unchanged / total_transitions) * 100
                
                print(f"  {version}:")
                print(f"    • Improvements: {improvement_rate:.1f}% ({improvements}/{total_transitions})")
                print(f"    • Degradations: {degradation_rate:.1f}% ({degradations}/{total_transitions})")
                print(f"    • Stable: {stability_rate:.1f}% ({unchanged}/{total_transitions})")
    
    # Net change analysis
    print(f"\n⚖️  NET RATING CHANGES:")
    print("-"*30)
    
    net_changes = {}
    for version, results in stuart_maxwell_results.items():
        matrix = results['transition_matrix']
        
        # Calculate net change for each rating category
        neg1_net = (matrix[:, 0].sum() - matrix[0, :].sum())  # Net flow into -1
        zero_net = (matrix[:, 1].sum() - matrix[1, :].sum())  # Net flow into 0  
        pos1_net = (matrix[:, 2].sum() - matrix[2, :].sum())  # Net flow into +1
        
        net_changes[version] = {
            'neg1_net': neg1_net,
            'zero_net': zero_net, 
            'pos1_net': pos1_net
        }
        
        print(f"{version}:")
        print(f"  • Net change to -1: {neg1_net:+d}")
        print(f"  • Net change to  0: {zero_net:+d}")
        print(f"  • Net change to +1: {pos1_net:+d}")
    
    # Overall insights
    print(f"\n💡 KEY INSIGHTS:")
    print("="*25)
    
    if valid_results:
        avg_test_stat = np.mean([r['test_statistic'] for r in valid_results.values()])
        significant_count = len([v for v, r in valid_results.items() 
                               if not np.isnan(r['p_value']) and r['p_value'] < 0.05])
        
        if significant_count > len(valid_results) / 2:
            print("🔄 MAJOR FINDING: Most versions show significant rating asymmetries")
            print("   → Verification process systematically changes rating distributions")
        elif significant_count > 0:
            print("📊 MIXED FINDINGS: Some versions show rating asymmetries")
            print("   → Version-specific effects on rating patterns")
        else:
            print("✅ BALANCED FINDINGS: No strong evidence of systematic rating bias")
            print("   → Verification maintains rating distributions well")
        
        # Best performing version
        best_balance = min(valid_results.items(), key=lambda x: x[1]['test_statistic'])
        print(f"\n🏆 Most balanced rating transitions: {best_balance[0]}")
        print(f"   → Lowest asymmetry score: {best_balance[1]['test_statistic']:.3f}")
    
    return stuart_maxwell_results, net_changes

# Run Stuart-Maxwell analysis
stuart_maxwell_results, net_changes = create_stuart_maxwell_analysis(combined_df)

## STUART-MAXWELL TEST REPORT GENERATOR

In [ ]:
def generate_stuart_maxwell_report(stuart_maxwell_results, net_changes, save_to_file=False, filename='stuart_maxwell_report.txt'):
    """
    Generate a comprehensive text report of Stuart-Maxwell test analysis for thesis documentation
    """
    
    report = []
    report.append("=" * 80)
    report.append("STUART-MAXWELL TEST ANALYSIS REPORT")
    report.append("Marginal Homogeneity Testing for Rating Transition Matrices")
    report.append("=" * 80)
    report.append("")
    
    # Executive Summary
    report.append("EXECUTIVE SUMMARY")
    report.append("-" * 50)
    
    valid_results = {v: r for v, r in stuart_maxwell_results.items() 
                    if not np.isnan(r['test_statistic'])}
    
    if len(valid_results) > 0:
        # Calculate summary statistics
        significant_versions = [v for v, r in valid_results.items() 
                              if not np.isnan(r['p_value']) and r['p_value'] < 0.05]
        
        avg_test_stat = np.mean([r['test_statistic'] for r in valid_results.values()])
        total_versions = len(valid_results)
        significant_count = len(significant_versions)
        
        report.append(f"Total versions analyzed: {total_versions}")
        report.append(f"Versions with significant asymmetry (p < 0.05): {significant_count}/{total_versions} ({(significant_count/total_versions)*100:.1f}%)")
        report.append(f"Average test statistic across versions: {avg_test_stat:.3f}")
        report.append("")
        
        # Overall interpretation
        if significant_count > total_versions / 2:
            report.append("OVERALL FINDING: SYSTEMATIC RATING BIAS DETECTED")
            report.append("The majority of versions show statistically significant evidence of")
            report.append("non-symmetric rating transitions, indicating systematic bias in the")
            report.append("verification process.")
        elif significant_count > 0:
            report.append("OVERALL FINDING: MIXED EVIDENCE OF RATING BIAS")
            report.append("Some versions show evidence of rating asymmetries while others")
            report.append("maintain balanced transitions.")
        else:
            report.append("OVERALL FINDING: NO SYSTEMATIC RATING BIAS")
            report.append("All versions maintain marginal homogeneity, indicating balanced")
            report.append("rating transitions without systematic bias.")
        
        report.append("")
    
    # Individual Version Results
    report.append("DETAILED VERSION ANALYSIS")
    report.append("-" * 50)
    
    for version, results in stuart_maxwell_results.items():
        report.append(f"{version.upper()}")
        report.append("─" * len(version))
        
        # Basic statistics
        report.append(f"Sample size: {results['sample_size']} transitions")
        
        if not np.isnan(results['test_statistic']):
            report.append(f"Stuart-Maxwell χ² statistic: {results['test_statistic']:.4f}")
            report.append(f"Degrees of freedom: 2")
            report.append(f"P-value: {results['p_value']:.4f}")
            
            # Significance interpretation
            if results['p_value'] < 0.001:
                significance_level = "p < 0.001 (***)"
                interpretation = "Highly significant evidence against marginal homogeneity"
            elif results['p_value'] < 0.01:
                significance_level = "p < 0.01 (**)"
                interpretation = "Strong evidence against marginal homogeneity"
            elif results['p_value'] < 0.05:
                significance_level = "p < 0.05 (*)"
                interpretation = "Significant evidence against marginal homogeneity"
            else:
                significance_level = "p ≥ 0.05 (ns)"
                interpretation = "No significant evidence against marginal homogeneity"
            
            report.append(f"Significance: {significance_level}")
            report.append(f"Interpretation: {interpretation}")
            
        else:
            report.append(f"Test status: {results['status']}")
            report.append("Unable to compute test statistic")
        
        # Marginal distributions
        matrix = results['transition_matrix']
        row_marginals = matrix.sum(axis=1)
        col_marginals = matrix.sum(axis=0)
        
        report.append("")
        report.append("Transition Matrix Details:")
        report.append("  From -1 → To -1: {:4d}   From -1 → To  0: {:4d}   From -1 → To +1: {:4d}".format(
            int(matrix[0, 0]), int(matrix[0, 1]), int(matrix[0, 2])))
        report.append("  From  0 → To -1: {:4d}   From  0 → To  0: {:4d}   From  0 → To +1: {:4d}".format(
            int(matrix[1, 0]), int(matrix[1, 1]), int(matrix[1, 2])))
        report.append("  From +1 → To -1: {:4d}   From +1 → To  0: {:4d}   From +1 → To +1: {:4d}".format(
            int(matrix[2, 0]), int(matrix[2, 1]), int(matrix[2, 2])))
        
        # Calculate percentages for each transition
        total_transitions = matrix.sum()
        if total_transitions > 0:
            report.append("")
            report.append("Transition Percentages:")
            for i, from_rating in enumerate(['-1', ' 0', '+1']):
                for j, to_rating in enumerate(['-1', ' 0', '+1']):
                    count = int(matrix[i, j])
                    percentage = (count / total_transitions) * 100
                    report.append("  From {} → To {}: {:4d} ({:5.1f}%)".format(
                        from_rating, to_rating, count, percentage))
        
        # Row-wise percentages (conditional probabilities)
        report.append("")
        report.append("Conditional Transition Probabilities (by original rating):")
        for i, from_rating in enumerate(['-1', ' 0', '+1']):
            row_total = row_marginals[i]
            if row_total > 0:
                report.append("  Given original rating {}:".format(from_rating))
                for j, to_rating in enumerate(['-1', ' 0', '+1']):
                    count = int(matrix[i, j])
                    prob = (count / row_total) * 100
                    report.append("    → To {}: {:4d}/{:4d} ({:5.1f}%)".format(
                        to_rating, count, int(row_total), prob))
            else:
                report.append("  Given original rating {}: No data".format(from_rating))
        
        report.append("")
        report.append("Marginal Distributions:")
        report.append(f"  Original ratings:  -1: {int(row_marginals[0])}, 0: {int(row_marginals[1])}, +1: {int(row_marginals[2])}")
        report.append(f"  Verified ratings:  -1: {int(col_marginals[0])}, 0: {int(col_marginals[1])}, +1: {int(col_marginals[2])}")
        
        # Net changes
        if version in net_changes:
            net = net_changes[version]
            report.append("")
            report.append("Net Rating Changes:")
            report.append(f"  Net flow to -1: {net['neg1_net']:+d}")
            report.append(f"  Net flow to  0: {net['zero_net']:+d}")
            report.append(f"  Net flow to +1: {net['pos1_net']:+d}")
            
            # Interpret net changes
            total_net_change = abs(net['neg1_net']) + abs(net['zero_net']) + abs(net['pos1_net'])
            if total_net_change > 0:
                if net['pos1_net'] > 0:
                    report.append("  → Overall tendency: Ratings improve (positive bias)")
                elif net['neg1_net'] > 0:
                    report.append("  → Overall tendency: Ratings degrade (negative bias)")
                else:
                    report.append("  → Overall tendency: Ratings shift toward neutral")
            else:
                report.append("  → Overall tendency: Perfectly balanced transitions")
        
        # Transition pattern analysis
        total_transitions = matrix.sum()
        if total_transitions > 0:
            improvements = matrix[0, 1] + matrix[0, 2] + matrix[1, 2]
            degradations = matrix[1, 0] + matrix[2, 0] + matrix[2, 1]
            unchanged = matrix[0, 0] + matrix[1, 1] + matrix[2, 2]
            
            improvement_rate = (improvements / total_transitions) * 100
            degradation_rate = (degradations / total_transitions) * 100
            stability_rate = (unchanged / total_transitions) * 100
            
            report.append("")
            report.append("Transition Patterns:")
            report.append(f"  Improvements: {improvement_rate:.1f}% ({improvements}/{total_transitions})")
            report.append(f"  Degradations: {degradation_rate:.1f}% ({degradations}/{total_transitions})")
            report.append(f"  Unchanged:    {stability_rate:.1f}% ({unchanged}/{total_transitions})")
        
        report.append("")
        report.append("")
    
    # Cross-version comparison
    if len(valid_results) >= 2:
        report.append("CROSS-VERSION COMPARISON")
        report.append("-" * 50)
        
        # Rank versions by test statistic
        version_rankings = sorted(valid_results.items(), key=lambda x: x[1]['test_statistic'])
        
        report.append("Version Rankings (by asymmetry level, low to high):")
        for rank, (version, results) in enumerate(version_rankings, 1):
            status = "Symmetric" if results['p_value'] >= 0.05 else "Asymmetric"
            report.append(f"  {rank}. {version}: χ² = {results['test_statistic']:.3f} ({status})")
        
        report.append("")
        
        # Best and worst performers
        best_version, best_results = version_rankings[0]
        worst_version, worst_results = version_rankings[-1]
        
        report.append(f"Most balanced transitions: {best_version}")
        report.append(f"  χ² = {best_results['test_statistic']:.3f}, p = {best_results['p_value']:.4f}")
        report.append("")
        report.append(f"Most asymmetric transitions: {worst_version}")
        report.append(f"  χ² = {worst_results['test_statistic']:.3f}, p = {worst_results['p_value']:.4f}")
        report.append("")
        
        # Effect size analysis
        effect_sizes = []
        for version, results in valid_results.items():
            # Calculate effect size (Cramér's V approximation)
            n = results['sample_size']
            chi_sq = results['test_statistic']
            if n > 0:
                cramers_v = np.sqrt(chi_sq / (n * 2))  # 2 = min(rows-1, cols-1) for 3x3
                effect_sizes.append((version, cramers_v))
        
        if effect_sizes:
            effect_sizes.sort(key=lambda x: x[1])
            report.append("Effect Sizes (Cramér's V approximation):")
            for version, effect_size in effect_sizes:
                if effect_size < 0.1:
                    magnitude = "Negligible"
                elif effect_size < 0.3:
                    magnitude = "Small"
                elif effect_size < 0.5:
                    magnitude = "Medium"
                else:
                    magnitude = "Large"
                report.append(f"  {version}: {effect_size:.3f} ({magnitude})")
            report.append("")
    
    # Statistical interpretation guide
    report.append("STATISTICAL INTERPRETATION GUIDE")
    report.append("-" * 50)
    report.append("Stuart-Maxwell Test:")
    report.append("  H₀: Marginal distributions are homogeneous (no systematic bias)")
    report.append("  H₁: Marginal distributions are not homogeneous (systematic bias exists)")
    report.append("")
    report.append("Significance Levels:")
    report.append("  p < 0.001 (***): Highly significant evidence of bias")
    report.append("  p < 0.01  (**):  Strong evidence of bias")
    report.append("  p < 0.05  (*):   Significant evidence of bias")
    report.append("  p ≥ 0.05  (ns):  No significant evidence of bias")
    report.append("")
    report.append("Effect Size Interpretation (Cramér's V):")
    report.append("  < 0.1: Negligible effect")
    report.append("  0.1-0.3: Small effect")
    report.append("  0.3-0.5: Medium effect")
    report.append("  > 0.5: Large effect")
    report.append("")
    
    # Practical implications
    report.append("PRACTICAL IMPLICATIONS")
    report.append("-" * 50)
    
    if len(valid_results) > 0:
        if significant_count > total_versions / 2:
            report.append("RECOMMENDATION: SYSTEMATIC BIAS CORRECTION NEEDED")
            report.append("• Multiple versions show significant rating asymmetries")
            report.append("• Consider implementing bias correction mechanisms")
            report.append("• Investigate root causes of systematic rating shifts")
            report.append(f"• Use {best_version} as benchmark for balanced transitions")
            
        elif significant_count > 0:
            report.append("RECOMMENDATION: VERSION-SPECIFIC OPTIMIZATION")
            report.append("• Some versions maintain balanced transitions while others show bias")
            report.append(f"• Prioritize deployment of balanced versions: {', '.join([v for v, r in version_rankings[:2]])}")
            report.append(f"• Investigate and correct biased versions: {', '.join(significant_versions)}")
            
        else:
            report.append("RECOMMENDATION: MAINTAIN CURRENT APPROACH")
            report.append("• All versions demonstrate balanced rating transitions")
            report.append("• No evidence of systematic bias across the verification system")
            report.append("• Continue monitoring for potential bias development")
        
        report.append("")
        report.append("Quality Assurance Guidelines:")
        report.append("• Monitor transition matrices regularly for bias development")
        report.append("• Set up alerts for p-values < 0.05 in production systems")
        report.append("• Consider periodic recalibration if systematic bias emerges")
        report.append(f"• Use {best_version} configuration as the gold standard")
    
    report.append("")
    report.append("=" * 80)
    report.append("END OF STUART-MAXWELL TEST ANALYSIS REPORT")
    report.append("=" * 80)
    
    # Convert to string
    report_text = "\n".join(report)
    
    # Print to console
    print(report_text)
    
    # Save to file if requested
    if save_to_file:
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(report_text)
            print(f"\n✅ Report saved to {filename}")
        except Exception as e:
            print(f"\n❌ Error saving report: {e}")
    
    return report_text

# Generate the Stuart-Maxwell report
if stuart_maxwell_results is not None:
    stuart_maxwell_report = generate_stuart_maxwell_report(
        stuart_maxwell_results, 
        net_changes, 
        save_to_file=False,  # Set to True to save to file
        filename='stuart_maxwell_thesis_report.txt'
    )
else:
    print("❌ Stuart-Maxwell analysis results not available")

## LLM Calls & Correlation

In [ ]:
# Generate the reports
if correlation_results is not None:
    correlation_report = generate_correlation_heatmap_report(
        correlation_results, 
        combined_df,
        save_to_file=False,  # Set to True to save to file
        filename='correlation_heatmap_thesis_report.txt'
    )
else:
    print("❌ Correlation analysis results not available")

print("\n" + "="*80 + "\n")

if steps_llm_results is not None:
    steps_llm_report = generate_steps_llm_relationship_report(
        steps_llm_results,
        combined_df,
        save_to_file=False,  # Set to True to save to file
        filename='steps_llm_relationship_thesis_report.txt'
    )
else:
    print("❌ Steps vs LLM calls analysis results not available")

## RECOMMENDATIONS

In [ ]:
def statistical_analysis_and_recommendations(combined_df, ranking_results):
    """
    Perform statistical analysis and generate actionable recommendations
    """
    print("\n" + "="*70)
    print("📊 STATISTICAL ANALYSIS & RECOMMENDATIONS")
    print("="*70)
    
    versions = combined_df['version'].unique()
    
    # Statistical significance testing
    print("🔬 STATISTICAL SIGNIFICANCE TESTS:")
    print("-" * 50)
    
    # ANOVA tests for continuous variables
    continuous_vars = ['processing_time_ms', 'total_llm_calls', 'token_efficiency', 'rating_improvement']
    
    for var in continuous_vars:
        if var in combined_df.columns:
            groups = []
            group_names = []
            for version in versions:
                version_data = combined_df[combined_df['version'] == version][var].dropna()
                if len(version_data) > 1:  # Need at least 2 data points
                    groups.append(version_data)
                    group_names.append(version)
            
            if len(groups) >= 2:
                try:
                    # Check if there's any variance in the data
                    all_data = np.concatenate(groups)
                    if np.var(all_data) > 1e-10:  # Avoid division by zero
                        f_stat, p_value = stats.f_oneway(*groups)
                        significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
                        print(f"  {var}: F={f_stat:.3f}, p={p_value:.4f} {significance}")
                    else:
                        print(f"  {var}: No variance detected (all values approximately equal)")
                except Exception as e:
                    print(f"  {var}: ANOVA failed ({str(e)[:30]}...)")
            else:
                print(f"  {var}: Insufficient data for ANOVA (need ≥2 groups with ≥2 data points each)")
    
    # Effect size analysis (Cohen's d between best and worst versions)
    best_version = ranking_results['final_ranking'][0][0]
    worst_version = ranking_results['final_ranking'][-1][0]
    
    print(f"\n📏 EFFECT SIZE ANALYSIS ({best_version} vs {worst_version}):")
    print("-" * 50)
    
    def cohens_d(group1, group2):
        """Calculate Cohen's d effect size with error handling"""
        try:
            if len(group1) < 2 or len(group2) < 2:
                return np.nan
            
            n1, n2 = len(group1), len(group2)
            s1, s2 = group1.std(), group2.std()
            
            # Handle case where standard deviations are zero
            if s1 == 0 and s2 == 0:
                return 0.0 if group1.mean() == group2.mean() else np.inf
            
            pooled_std = np.sqrt(((n1-1)*s1**2 + (n2-1)*s2**2) / (n1+n2-2))
            
            if pooled_std == 0:
                return 0.0 if group1.mean() == group2.mean() else np.inf
            
            return (group1.mean() - group2.mean()) / pooled_std
        except Exception as e:
            return np.nan
    
    for var in continuous_vars:
        if var in combined_df.columns:
            best_data = combined_df[combined_df['version'] == best_version][var].dropna()
            worst_data = combined_df[combined_df['version'] == worst_version][var].dropna()
            
            if len(best_data) > 0 and len(worst_data) > 0:
                effect_size = cohens_d(best_data, worst_data)
                magnitude = ("Large" if abs(effect_size) >= 0.8 else 
                           "Medium" if abs(effect_size) >= 0.5 else "Small")
                print(f"  {var}: Cohen's d = {effect_size:.3f} ({magnitude})")
    
    # Confidence intervals for key metrics
    print(f"\n🎯 95% CONFIDENCE INTERVALS FOR KEY METRICS:")
    print("-" * 50)
    
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        print(f"\n{version}:")
        
        # Success rate CI
        n = len(version_data)
        p = version_data['success'].mean()
        se = np.sqrt(p * (1-p) / n)
        ci_lower = p - 1.96 * se
        ci_upper = p + 1.96 * se
        print(f"  Success Rate: {p*100:.1f}% [{ci_lower*100:.1f}%, {ci_upper*100:.1f}%]")
        
        # Processing time CI
        if 'processing_time_ms' in version_data.columns:
            times = version_data['processing_time_ms'].dropna()
            if len(times) > 1:
                mean_time = times.mean()
                se_time = times.std() / np.sqrt(len(times))
                ci_lower_time = mean_time - 1.96 * se_time
                ci_upper_time = mean_time + 1.96 * se_time
                print(f"  Processing Time: {mean_time:.1f}ms [{ci_lower_time:.1f}ms, {ci_upper_time:.1f}ms]")
    
    # Generate recommendations
    print(f"\n🎯 ACTIONABLE RECOMMENDATIONS:")
    print("="*50)
    
    recommendations = []
    
    # Best performer analysis
    best_version = ranking_results['final_ranking'][0][0]
    best_score = ranking_results['final_ranking'][0][1]
    
    recommendations.append(f"🥇 ADOPT BEST PRACTICES: {best_version} achieved the highest overall score ({best_score:.1f}/100)")
    
    # Identify strongest metrics per version
    metric_winners = {}
    for metric in ranking_results['metric_rankings']:
        winner = ranking_results['metric_rankings'][metric][0]
        metric_winners[metric] = winner
    
    print(f"\n🏆 METRIC CHAMPIONS:")
    for metric, (version, score) in metric_winners.items():
        print(f"  • {metric.replace('_', ' ').title()}: {version} ({score:.1f}/100)")
        
        if metric == 'processing_speed':
            recommendations.append(f"⚡ SPEED OPTIMIZATION: Learn from {version}'s processing optimizations")
        elif metric == 'accuracy_improvement':
            recommendations.append(f"🎯 ACCURACY FOCUS: Adopt {version}'s verification strategies")
        elif metric == 'token_efficiency':
            recommendations.append(f"💰 COST OPTIMIZATION: Implement {version}'s token efficiency techniques")
    
    # Identify improvement opportunities
    print(f"\n🔍 IMPROVEMENT OPPORTUNITIES:")
    
    for version in versions:
        version_scores = {metric: ranking_results['normalized_data'][version][metric] 
                         for metric in ranking_results['normalized_data'][version]}
        weakest_metric = min(version_scores, key=version_scores.get)
        weakest_score = version_scores[weakest_metric]
        
        if weakest_score < 50:  # Below average performance
            print(f"  • {version}: Improve {weakest_metric.replace('_', ' ')} (current: {weakest_score:.1f}/100)")
            recommendations.append(f"📈 {version} FOCUS: Prioritize {weakest_metric.replace('_', ' ')} improvements")
    
    # Version-specific insights
    print(f"\n💡 VERSION-SPECIFIC INSIGHTS:")
    
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        insights = []
        
        # Success rate insight
        success_rate = version_data['success'].mean() * 100
        if success_rate >= 95:
            insights.append("Very high reliability")
        elif success_rate < 80:
            insights.append("Reliability concerns")
        
        # Processing time insight
        avg_time = version_data['processing_time_ms'].mean()
        if avg_time < 50:
            insights.append("Fast processing")
        elif avg_time > 100:
            insights.append("Slow processing")
        
        # Token efficiency insight
        token_eff = version_data['token_efficiency'].mean()
        if token_eff > 0.3:
            insights.append("High token efficiency")
        elif token_eff < 0.15:
            insights.append("Low token efficiency")
        
        if insights:
            print(f"  • {version}: {', '.join(insights)}")
    
    # Final recommendations
    print(f"\n📋 FINAL RECOMMENDATIONS:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
    
    # Strategic recommendations
    strategic_recs = [
        "🔄 Implement A/B testing framework to continuously evaluate version performance",
        "📊 Set up monitoring dashboards to track key metrics in real-time", 
        "🎯 Focus development efforts on the weakest performing metrics",
        "🔬 Conduct detailed analysis of failure cases to improve reliability",
        "💰 Optimize token usage based on best-performing version's strategies"
    ]
    
    print(f"\n🚀 STRATEGIC RECOMMENDATIONS:")
    for i, rec in enumerate(strategic_recs, 1):
        print(f"{i}. {rec}")

    return {
        'recommendations': recommendations,
        'strategic_recommendations': strategic_recs,
        'metric_winners': metric_winners
    }

# Run statistical analysis and recommendations
analysis_recommendations = statistical_analysis_and_recommendations(combined_df, ranking_results)

# %%
# =============================================================================
# EXPORT AND SUMMARY FUNCTIONS
# =============================================================================

def create_executive_summary(combined_df, ranking_results, analysis_recommendations):
    """
    Create an executive summary for stakeholders
    """
    print("\n" + "="*80)
    print("📋 EXECUTIVE SUMMARY - VERIFICATION SYSTEM VERSION COMPARISON")
    print("="*80)
    
    # Key findings
    best_version = ranking_results['final_ranking'][0][0]
    best_score = ranking_results['final_ranking'][0][1]
    worst_version = ranking_results['final_ranking'][-1][0]
    worst_score = ranking_results['final_ranking'][-1][1]
    
    total_problems = len(combined_df)
    num_versions = len(combined_df['version'].unique())
    
    print(f"📊 ANALYSIS OVERVIEW:")
    print(f"   • Total Problems Analyzed: {total_problems:,}")
    print(f"   • Number of Versions: {num_versions}")
    print(f"   • Analysis Date: June 2025")
    
    print(f"\n🏆 KEY FINDINGS:")
    print(f"   • Best Performing Version: {best_version} (Score: {best_score:.1f}/100)")
    print(f"   • Lowest Performing Version: {worst_version} (Score: {worst_score:.1f}/100)")
    print(f"   • Performance Gap: {best_score - worst_score:.1f} points")
    
    # Metric champions
    print(f"\n🥇 METRIC LEADERS:")
    for metric, (version, score) in analysis_recommendations['metric_winners'].items():
        print(f"   • {metric.replace('_', ' ').title()}: {version}")
    
    # Overall statistics
    overall_stats = combined_df.groupby('version').agg({
        'success': 'mean',
        'processing_time_ms': 'mean',
        'total_llm_calls': 'mean'
    }).round(3)
    
    print(f"\n📈 PERFORMANCE SUMMARY:")
    print(f"   • Average Success Rate: {combined_df['success'].mean()*100:.1f}%")
    print(f"   • Average Processing Time: {combined_df['processing_time_ms'].mean():.1f}ms")
    print(f"   • Average LLM Calls: {combined_df['total_llm_calls'].mean():.1f}")
    
    # Recommendations summary
    print(f"\n💡 TOP 3 RECOMMENDATIONS:")
    top_recs = analysis_recommendations['recommendations'][:3]
    for i, rec in enumerate(top_recs, 1):
        print(f"   {i}. {rec}")

def export_multi_version_results(combined_df, ranking_results, analysis_recommendations, 
                                filename_prefix='multi_version_analysis'):
    """
    Export comprehensive multi-version analysis results
    """
    
    # 1. Export summary JSON
    summary_data = {
        'analysis_info': {
            'total_problems': len(combined_df),
            'num_versions': len(combined_df['version'].unique()),
            'analysis_date': '2025-06-17'
        },
        'version_ranking': {
            'final_ranking': ranking_results['final_ranking'],
            'metric_rankings': ranking_results['metric_rankings']
        },
        'recommendations': {
            'actionable_recommendations': analysis_recommendations['recommendations'],
            'strategic_recommendations': analysis_recommendations['strategic_recommendations'],
            'metric_winners': analysis_recommendations['metric_winners']
        },
        'version_statistics': {}
    }
    
    # Add version statistics
    for version in combined_df['version'].unique():
        version_data = combined_df[combined_df['version'] == version]
        summary_data['version_statistics'][version] = {
            'total_problems': len(version_data),
            'success_rate': version_data['success'].mean(),
            'avg_processing_time': version_data['processing_time_ms'].mean(),
            'avg_llm_calls': version_data['total_llm_calls'].mean(),
            'avg_token_efficiency': version_data['token_efficiency'].mean()
        }
    
    # Convert numpy types for JSON serialization
    def convert_numpy_types(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(item) for item in obj]
        return obj
    
    summary_data = convert_numpy_types(summary_data)
    
    try:
        with open(f'{filename_prefix}_summary.json', 'w') as f:
            json.dump(summary_data, f, indent=2, default=str)
        print(f"✅ Summary exported to {filename_prefix}_summary.json")
    except Exception as e:
        print(f"❌ Error exporting summary: {e}")
    
    # 2. Export detailed CSV
    try:
        combined_df.to_csv(f'{filename_prefix}_detailed.csv', index=False)
        print(f"✅ Detailed data exported to {filename_prefix}_detailed.csv")
    except Exception as e:
        print(f"❌ Error exporting detailed data: {e}")
    
    # 3. Export ranking table
    try:
        ranking_df = pd.DataFrame([
            {
                'Rank': rank,
                'Version': version,
                'Overall_Score': score,
                **ranking_results['raw_data'][version]
            }
            for rank, (version, score) in enumerate(ranking_results['final_ranking'], 1)
        ])
        
        ranking_df.to_csv(f'{filename_prefix}_ranking.csv', index=False)
        print(f"✅ Ranking table exported to {filename_prefix}_ranking.csv")
    except Exception as e:
        print(f"❌ Error exporting ranking: {e}")

# Create executive summary
create_executive_summary(combined_df, ranking_results, analysis_recommendations)

# %%
# Final Comparison Dashboard
def create_final_comparison_dashboard(combined_df, ranking_results):
    """
    Create a comprehensive final dashboard for version comparison
    """
    fig = plt.figure(figsize=(20, 12))
    fig.suptitle('🎯 FINAL VERSION COMPARISON DASHBOARD', fontsize=20, fontweight='bold')
    
    # Create a complex grid layout
    gs = fig.add_gridspec(3, 4, height_ratios=[1, 1, 1], width_ratios=[1, 1, 1, 1])
    
    versions = combined_df['version'].unique()
    colors = plt.cm.Set2(np.linspace(0, 1, len(versions)))
    
    # 1. Overall Ranking (Top Left)
    ax1 = fig.add_subplot(gs[0, 0])
    ranking_scores = [score for _, score in ranking_results['final_ranking']]
    ranking_versions = [version for version, _ in ranking_results['final_ranking']]
    
    bars = ax1.barh(range(len(ranking_versions)), ranking_scores, color=colors)
    ax1.set_yticks(range(len(ranking_versions)))
    ax1.set_yticklabels(ranking_versions)
    ax1.set_xlabel('Overall Score')
    ax1.set_title('🏆 Final Rankings', fontweight='bold')
    ax1.invert_yaxis()
    
    for i, (bar, score) in enumerate(zip(bars, ranking_scores)):
        ax1.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                f'{score:.1f}', va='center', fontweight='bold')
    
    # 2. Success Rate Comparison (Top Middle)
    ax2 = fig.add_subplot(gs[0, 1])
    success_rates = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        success_rates.append(version_data['success'].mean() * 100)
    
    bars2 = ax2.bar(versions, success_rates, color=colors)
    ax2.set_ylabel('Success Rate (%)')
    ax2.set_title('✅ Success Rates', fontweight='bold')
    ax2.tick_params(axis='x', rotation=45)
    
    for bar, value in zip(bars2, success_rates):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    # 3. Processing Time Box Plot (Top Right)
    ax3 = fig.add_subplot(gs[0, 2])
    time_data = []
    time_labels = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        times = version_data['processing_time_ms'].dropna()
        if len(times) > 0:
            time_data.append(times)
            time_labels.append(version.replace('Version ', 'V'))
    
    if time_data:
        bp = ax3.boxplot(time_data, labels=time_labels, patch_artist=True)
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
    ax3.set_ylabel('Processing Time (ms)')
    ax3.set_title('⏱️ Processing Time', fontweight='bold')
    ax3.tick_params(axis='x', rotation=45)
    
    # 4. Token Efficiency (Top Far Right)
    ax4 = fig.add_subplot(gs[0, 3])
    efficiency_data = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        efficiency_data.append(version_data['token_efficiency'].mean())
    
    bars4 = ax4.bar(versions, efficiency_data, color=colors)
    ax4.set_ylabel('Token Efficiency')
    ax4.set_title('💰 Token Efficiency', fontweight='bold')
    ax4.tick_params(axis='x', rotation=45)
    
    for bar, value in zip(bars4, efficiency_data):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 5. Performance Radar Chart (Middle Left & Middle)
    ax5 = fig.add_subplot(gs[1, :2])
    
    metrics = ['Success\nRate', 'Accuracy\nImprovement', 'Processing\nSpeed', 'Token\nEfficiency', 'Rating\nImprovement']
    
    # Normalize data for radar chart
    normalized_scores = {}
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        
        success_norm = version_data['success'].mean() * 100
        acc_imp = (version_data['verifier_answer_correctness'].mean() - 
                   version_data['original_answer_correctness'].mean()) * 100
        acc_imp_norm = max(0, min(100, acc_imp + 50))  # Shift to positive scale
        speed_norm = max(0, min(100, 100 - (version_data['processing_time_ms'].mean() / 2)))
        efficiency_norm = min(100, version_data['token_efficiency'].mean() * 300)
        rating_imp = version_data['rating_improvement'].dropna().mean()
        rating_norm = max(0, min(100, (rating_imp * 100) + 50)) if not pd.isna(rating_imp) else 50
        
        normalized_scores[version] = [success_norm, acc_imp_norm, speed_norm, efficiency_norm, rating_norm]
    
    # Create grouped bar chart instead of radar for better readability
    x_pos = np.arange(len(metrics))
    width = 0.15
    
    for i, (version, scores) in enumerate(normalized_scores.items()):
        ax5.bar(x_pos + i * width, scores, width, label=version, color=colors[i], alpha=0.8)
    
    ax5.set_xlabel('Performance Metrics')
    ax5.set_ylabel('Normalized Score (0-100)')
    ax5.set_title('📊 Multi-Metric Performance Comparison', fontweight='bold')
    ax5.set_xticks(x_pos + width * 2)
    ax5.set_xticklabels(metrics)
    ax5.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax5.grid(axis='y', alpha=0.3)
    
    # 6. Key Insights Text (Middle Right & Far Right)
    ax6 = fig.add_subplot(gs[1, 2:])
    ax6.axis('off')
    
    # Generate key insights text
    best_version = ranking_results['final_ranking'][0][0]
    best_score = ranking_results['final_ranking'][0][1]
    
    insights_text = f"""
🏆 WINNER: {best_version}
Overall Score: {best_score:.1f}/100

🎯 KEY INSIGHTS:
• Best Success Rate: {max(success_rates):.1f}%
• Fastest Average Time: {min([combined_df[combined_df['version'] == v]['processing_time_ms'].mean() for v in versions]):.1f}ms
• Most Efficient: {max(efficiency_data):.3f} tokens

📈 RECOMMENDATIONS:
• Adopt {best_version}'s strategies
• Focus on weakest metrics
• Implement continuous monitoring
• A/B test improvements

🚀 NEXT STEPS:
1. Deploy best version
2. Analyze failure patterns  
3. Optimize weak performers
4. Set up monitoring dashboard
"""
    
    ax6.text(0.05, 0.95, insights_text, transform=ax6.transAxes, fontsize=11,
             verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.3))
    
    # 7. Statistical Summary Table (Bottom)
    ax7 = fig.add_subplot(gs[2, :])
    ax7.axis('off')
    
    # Create summary statistics table
    summary_stats = []
    for version in versions:
        version_data = combined_df[combined_df['version'] == version]
        stats_row = [
            version,
            f"{len(version_data)}",
            f"{version_data['success'].mean()*100:.1f}%",
            f"{version_data['processing_time_ms'].mean():.1f}ms",
            f"{version_data['total_llm_calls'].mean():.1f}",
            f"{version_data['token_efficiency'].mean():.3f}",
            f"{ranking_results['raw_data'][version]['accuracy_improvement']:+.1f}pp"
        ]
        summary_stats.append(stats_row)
    
    # Create table
    table_data = summary_stats
    col_labels = ['Version', 'Problems', 'Success Rate', 'Avg Time', 'Avg LLM Calls', 'Token Efficiency', 'Accuracy Δ']
    
    table = ax7.table(cellText=table_data, colLabels=col_labels, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 2)
    
    # Color code the table based on ranking
    for i, (version, _) in enumerate(ranking_results['final_ranking']):
        row_idx = versions.tolist().index(version) + 1  # +1 for header
        if i == 0:  # Best performer
            for j in range(len(col_labels)):
                table[(row_idx, j)].set_facecolor('#90EE90')  # Light green
        elif i == len(ranking_results['final_ranking']) - 1:  # Worst performer
            for j in range(len(col_labels)):
                table[(row_idx, j)].set_facecolor('#FFB6C1')  # Light pink
    
    ax7.set_title('📋 Detailed Statistics Summary', fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()

create_final_comparison_dashboard(combined_df, ranking_results)

# %%
# Export results (uncomment to save files)
# export_multi_version_results(combined_df, ranking_results, analysis_recommendations)

print("\n🎉 MULTI-VERSION ANALYSIS COMPLETE!")
print("="*60)
print("✅ Comprehensive comparison of all 5 versions completed")
print("✅ Statistical analysis and recommendations generated")
print("✅ Performance rankings and insights provided")
print("✅ Executive summary and dashboard created")
print("\n📁 Uncomment export function to save detailed results")
print("🎯 Ready for implementation of recommendations!")

# %%
"""
MULTI-VERSION ANALYSIS SUMMARY:
===============================

This notebook provides comprehensive analysis of 5 verification system versions including:

📊 ANALYSIS COMPONENTS:
1. Multi-version data loading and preprocessing
2. Comparative foundational analysis with statistical testing
3. Comprehensive ranking system with weighted metrics
4. Performance heatmaps and visualizations
5. Statistical significance testing and effect size analysis
6. Actionable recommendations and strategic insights
7. Executive summary for stakeholders
8. Export functions for detailed reporting

🎯 KEY FEATURES:
• Automated ranking across multiple performance dimensions
• Statistical significance testing between versions
• Visual performance comparisons and heatmaps
• Confidence intervals for key metrics
• Actionable recommendations based on data
• Executive-ready summary and insights

🚀 USAGE:
1. Update file paths in load_multiple_versions() function
2. Run all cells sequentially
3. Review comparative analysis and rankings
4. Implement recommendations from best-performing versions
5. Use export functions to save results for stakeholders

📈 OUTPUTS:
• Version performance rankings
• Statistical analysis reports
• Comprehensive visualizations
• Actionable improvement recommendations
• Exportable summary data and detailed results
"""